# Credits
__This notebook has created starting from__
- https://git.skewed.de/count0/graph-tool/-/wikis/installation-instructions#googles-colaboratory

The installation procedure has been taken from here: https://colab.research.google.com/github/count0/colab-gt/blob/master/colab-gt.ipynb

In [ ]:
from google.colab import files

from google.colab import drive
drive.mount('/content/drive')

# Installing graph-tool (https://graph-tool.skewed.de)

---


To import a library that's not in Colaboratory by default, you can use `!apt-get install`.

But since graph-tool is not in the official repository, we need to add it to the list.

In [ ]:
!echo "deb http://downloads.skewed.de/apt focal main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo

In [ ]:
# Colab uses a Python install that deviates from the system's! Bad collab! We need some workarounds.
!apt purge python3-cairo
!apt install libcairo2-dev pkg-config python3-dev
!pip install --force-reinstall pycairo
!pip install zstandard

In [ ]:
import graph_tool.all as gt
model_name="blumenau_drug"
g = gt.collection.ns[model_name]
#g.get_edges()

# Repository of weighted graphs: https://networks.skewed.de/?tags=Weighted

In [1]:
import numpy as np
import pandas as pd
import graph_tool.all as gt
from google.colab import files

from google.colab import drive

import os.path
from os import path

import math   



def measure_allgraphs(model_names,download_file,store_on_gdrive,path_g_drive="weightedGraphsLinearODE/",prefix_to_weight=dict(),csv_file_name="csv_file"):
   #Counts edges and nodes of each graph.
   #The file is stored on google drive

  print('Processing and exporting models')
  print("",model_names)

  #Prepare gdrive
  local_file_path_to_import="WeightedGraph_measurer/"
  gdrive_path="/content/drive/MyDrive/"+path_g_drive
  if store_on_gdrive:
    drive.mount('/content/drive')
    if not path.exists(gdrive_path):
      print('creating',gdrive_path)
      os.mkdir(gdrive_path)
    #if not path.exists(gdrive_path+"/"+local_file_path_to_import):
    #  print('creating',gdrive_path+"/"+local_file_path_to_import)
    #  os.mkdir(gdrive_path+"/"+local_file_path_to_import)
    if not path.exists(gdrive_path+"/csv/"):
      os.mkdir(gdrive_path+"/csv/")
  
  csv_file_path_full=gdrive_path+"/csv/"+csv_file_name+'.csv'
  try:
      with open(csv_file_path_full, 'a') as csv_file: 
        #file.write('hello world !')
        if os.path.getsize(csv_file_path_full) <= 0:
          csv_file.write('name,weighted,edges,nodes,log_nodes,max_degree,mean_degree,min_degree\n')

        #Export required weighted graphs
        for model_name in model_names:
          weight_name='weight'
          for key in prefix_to_weight:
            if model_name.startswith(key):
              weight_name=prefix_to_weight[key]
              break
          measure_weightedGraph(model_name,download_file,store_on_gdrive,gdrive_path,weight_name,local_file_path_to_import,csv_file)
  finally:
    #Close gdrive
    if store_on_gdrive:
      drive.flush_and_unmount()

  print('All computations completed')

def measure_weightedGraph(model_name,download_file,store_on_gdrive,gdrive_path,weight_name,local_file_path_to_import,csv_file):
  #load model
  print("PROCESSING:",model_name,"...")
  
  g = gt.collection.ns[model_name]
    
  add_rev= not g.is_directed()
  weighted=True

  if not(weight_name in g.edge_properties):
    print(" MODEL NOT WEIGHTED")
    weighted=False
    #print(" MODEL DROPPED BECAUSE NOT WEIGHTED")
    #return
  
  n_nodes=g.num_vertices()
  n_edges=g.num_edges()
  if add_rev:
    n_edges*=2
  min_nodes=-1
  max_nodes=-1
  min_edges=-1
  max_edges=-1
  if max_nodes>0 and n_nodes > max_nodes:
    print(" MODEL DROPPED BECAUSE IT HAS MORE THAN",max_nodes,"NODES")
    return 
  elif min_nodes>0 and n_nodes <= min_nodes:
     print(" MODEL DROPPED BECAUSE IT HAS LESS THAN",min_nodes,"NODES")
     return
  elif max_edges>0 and n_edges > max_edges:
    print(" MODEL DROPPED BECAUSE IT HAS MORE THAN",max_edges,"EDGES")
    return 
  elif min_edges>0 and n_edges <= min_edges:
     print(" MODEL DROPPED BECAUSE IT HAS LESS THAN",min_edges,"EDGES")
     return
  else:
    print(" MODEL CONSIDERED")
  

  #name,weighted,edges,nodes,log_nodes,max_degree,mean_degree,min_degree
  degrees=g.get_total_degrees(g.get_vertices())
  #print(degrees.min(),degrees.mean(),degrees.max())
  
  csv_file.write(model_name)
  csv_file.write(',')    
  csv_file.write(str(weighted))
  csv_file.write(',')
  csv_file.write(str(n_edges))
  csv_file.write(',')
  csv_file.write(str(n_nodes))
  csv_file.write(',')
  csv_file.write(str(math.log(n_nodes)))
  csv_file.write(',')
  csv_file.write(str(degrees.max()))
  csv_file.write(',')
  csv_file.write(str(degrees.mean()))
  csv_file.write(',')
  csv_file.write(str(degrees.min()))
  csv_file.write('\n')
  csv_file.flush()
    
  print("  ","Graph processed, and row added to CSV")
    
  
#Choose the models to consider
# Just https://networks.skewed.de/net/blumenau_drug
model_names=["blumenau_drug"]
#model_names=["rhesus_monkey"]
#model_names=["ambassador/KINSHIP"]
#model_names=["dom/Archie_2006h"]
#model_names=["libimseti"]
#model_names=["twitter_higgs/mention"]
#model_names=["cattle"]
#model_names=["blumenau_drug","cattle","train_terrorists"]

#All weighted networks on 21/07/2022 from smallest number of nodes to largest#
# Taken from https://networks.skewed.de/?tags=Weighted after clicking on 'Nodes'
model_names=[
             "packet_delays","ambassador/1985_1989","ambassador/1990_1994","ambassador/1995_1999","ambassador/2000","ambassador/2001","ambassador/2002","ambassador/2003","ambassador/2004","ambassador/2005","ambassador/2006","ambassador/2007","ambassador/KINSHIP","ambassador/OPERATION_ID","ambassador/TIE_EXTINGUISH","ambassador/TIE_YEAR",
             "rhesus_monkey","kangaroo",
             "sp_baboons/observational","sp_baboons/sensor","bison","cattle","moreno_sheep","college_freshmen","hens",
             "freshmen/t0","freshmen/t2","freshmen/t3","freshmen/t5","freshmen/t6",
             "windsurfers","macaques","train_terrorists","highschool","blumenau_drug","lesmis","baseball/player-player","baseball/user-provider","sp_office","game_thrones",
             "moviegalaxies/1","moviegalaxies/2","moviegalaxies/3","moviegalaxies/5","moviegalaxies/6","moviegalaxies/7","moviegalaxies/8","moviegalaxies/9","moviegalaxies/10","moviegalaxies/11","moviegalaxies/13","moviegalaxies/14","moviegalaxies/15","moviegalaxies/17","moviegalaxies/19","moviegalaxies/21","moviegalaxies/22","moviegalaxies/23","moviegalaxies/24","moviegalaxies/25","moviegalaxies/26","moviegalaxies/27","moviegalaxies/28","moviegalaxies/30","moviegalaxies/31","moviegalaxies/32","moviegalaxies/33","moviegalaxies/34","moviegalaxies/36","moviegalaxies/38","moviegalaxies/39","moviegalaxies/40","moviegalaxies/41","moviegalaxies/42","moviegalaxies/43","moviegalaxies/44","moviegalaxies/45","moviegalaxies/47","moviegalaxies/48","moviegalaxies/49","moviegalaxies/50","moviegalaxies/51","moviegalaxies/52","moviegalaxies/54","moviegalaxies/57","moviegalaxies/58","moviegalaxies/59","moviegalaxies/60","moviegalaxies/61","moviegalaxies/62","moviegalaxies/63","moviegalaxies/64","moviegalaxies/65","moviegalaxies/66","moviegalaxies/67","moviegalaxies/68","moviegalaxies/69","moviegalaxies/70","moviegalaxies/71","moviegalaxies/72","moviegalaxies/73","moviegalaxies/74","moviegalaxies/76","moviegalaxies/80","moviegalaxies/81","moviegalaxies/82","moviegalaxies/83","moviegalaxies/84","moviegalaxies/85","moviegalaxies/86","moviegalaxies/87","moviegalaxies/88","moviegalaxies/89","moviegalaxies/91","moviegalaxies/92","moviegalaxies/93","moviegalaxies/97","moviegalaxies/98","moviegalaxies/99","moviegalaxies/100","moviegalaxies/101","moviegalaxies/102","moviegalaxies/103","moviegalaxies/104","moviegalaxies/106","moviegalaxies/107","moviegalaxies/108","moviegalaxies/109","moviegalaxies/110","moviegalaxies/111","moviegalaxies/112","moviegalaxies/113","moviegalaxies/114","moviegalaxies/115","moviegalaxies/116","moviegalaxies/117","moviegalaxies/118","moviegalaxies/119","moviegalaxies/120","moviegalaxies/122","moviegalaxies/123","moviegalaxies/124","moviegalaxies/125","moviegalaxies/126","moviegalaxies/127","moviegalaxies/128","moviegalaxies/129","moviegalaxies/130","moviegalaxies/131","moviegalaxies/132","moviegalaxies/133","moviegalaxies/134","moviegalaxies/135","moviegalaxies/136","moviegalaxies/137","moviegalaxies/138","moviegalaxies/139","moviegalaxies/141","moviegalaxies/142","moviegalaxies/143","moviegalaxies/144","moviegalaxies/145","moviegalaxies/146","moviegalaxies/147","moviegalaxies/148","moviegalaxies/149","moviegalaxies/150","moviegalaxies/151","moviegalaxies/152","moviegalaxies/153","moviegalaxies/154","moviegalaxies/155","moviegalaxies/157","moviegalaxies/158","moviegalaxies/159","moviegalaxies/161","moviegalaxies/162","moviegalaxies/163","moviegalaxies/164","moviegalaxies/165","moviegalaxies/166","moviegalaxies/167","moviegalaxies/169","moviegalaxies/170","moviegalaxies/171","moviegalaxies/173","moviegalaxies/174","moviegalaxies/175","moviegalaxies/176","moviegalaxies/177","moviegalaxies/178","moviegalaxies/179","moviegalaxies/180","moviegalaxies/182","moviegalaxies/183","moviegalaxies/184","moviegalaxies/185","moviegalaxies/186","moviegalaxies/187","moviegalaxies/189","moviegalaxies/190","moviegalaxies/191","moviegalaxies/192","moviegalaxies/193","moviegalaxies/194","moviegalaxies/195","moviegalaxies/197","moviegalaxies/198","moviegalaxies/199","moviegalaxies/200","moviegalaxies/201","moviegalaxies/204","moviegalaxies/205","moviegalaxies/206","moviegalaxies/208","moviegalaxies/209","moviegalaxies/210","moviegalaxies/211","moviegalaxies/212","moviegalaxies/213","moviegalaxies/214","moviegalaxies/216","moviegalaxies/217","moviegalaxies/218","moviegalaxies/220","moviegalaxies/221","moviegalaxies/222","moviegalaxies/223","moviegalaxies/224","moviegalaxies/225","moviegalaxies/226","moviegalaxies/227","moviegalaxies/229","moviegalaxies/231","moviegalaxies/232","moviegalaxies/234","moviegalaxies/235","moviegalaxies/236","moviegalaxies/238","moviegalaxies/239","moviegalaxies/240","moviegalaxies/241","moviegalaxies/242","moviegalaxies/243","moviegalaxies/244","moviegalaxies/246","moviegalaxies/247","moviegalaxies/248","moviegalaxies/249","moviegalaxies/250","moviegalaxies/251","moviegalaxies/252","moviegalaxies/253","moviegalaxies/254","moviegalaxies/255","moviegalaxies/256","moviegalaxies/257","moviegalaxies/258","moviegalaxies/260","moviegalaxies/261","moviegalaxies/262","moviegalaxies/263","moviegalaxies/264","moviegalaxies/265","moviegalaxies/266","moviegalaxies/268","moviegalaxies/269","moviegalaxies/271","moviegalaxies/272","moviegalaxies/273","moviegalaxies/274","moviegalaxies/276","moviegalaxies/278","moviegalaxies/279","moviegalaxies/280","moviegalaxies/281","moviegalaxies/282","moviegalaxies/283","moviegalaxies/286","moviegalaxies/287","moviegalaxies/288","moviegalaxies/289","moviegalaxies/290","moviegalaxies/291","moviegalaxies/292","moviegalaxies/293","moviegalaxies/294","moviegalaxies/295","moviegalaxies/296","moviegalaxies/297","moviegalaxies/298","moviegalaxies/299","moviegalaxies/300","moviegalaxies/301","moviegalaxies/302","moviegalaxies/303","moviegalaxies/304","moviegalaxies/305","moviegalaxies/306","moviegalaxies/307","moviegalaxies/308","moviegalaxies/309","moviegalaxies/310","moviegalaxies/311","moviegalaxies/313","moviegalaxies/314","moviegalaxies/315","moviegalaxies/316","moviegalaxies/317","moviegalaxies/318","moviegalaxies/319","moviegalaxies/320","moviegalaxies/321","moviegalaxies/322","moviegalaxies/323","moviegalaxies/324","moviegalaxies/325","moviegalaxies/326","moviegalaxies/327","moviegalaxies/328","moviegalaxies/329","moviegalaxies/330","moviegalaxies/331","moviegalaxies/332","moviegalaxies/333","moviegalaxies/334","moviegalaxies/335","moviegalaxies/336","moviegalaxies/337","moviegalaxies/338","moviegalaxies/339","moviegalaxies/340","moviegalaxies/341","moviegalaxies/343","moviegalaxies/344","moviegalaxies/346","moviegalaxies/347","moviegalaxies/348","moviegalaxies/349","moviegalaxies/350","moviegalaxies/351","moviegalaxies/352","moviegalaxies/354","moviegalaxies/355","moviegalaxies/356","moviegalaxies/357","moviegalaxies/358","moviegalaxies/359","moviegalaxies/361","moviegalaxies/362","moviegalaxies/363","moviegalaxies/364","moviegalaxies/366","moviegalaxies/368","moviegalaxies/369","moviegalaxies/370","moviegalaxies/371","moviegalaxies/372","moviegalaxies/373","moviegalaxies/374","moviegalaxies/375","moviegalaxies/376","moviegalaxies/377","moviegalaxies/378","moviegalaxies/379","moviegalaxies/383","moviegalaxies/384","moviegalaxies/385","moviegalaxies/386","moviegalaxies/387","moviegalaxies/388","moviegalaxies/389","moviegalaxies/391","moviegalaxies/393","moviegalaxies/394","moviegalaxies/395","moviegalaxies/396","moviegalaxies/397","moviegalaxies/398","moviegalaxies/399","moviegalaxies/400","moviegalaxies/401","moviegalaxies/402","moviegalaxies/403","moviegalaxies/404","moviegalaxies/405","moviegalaxies/406","moviegalaxies/407","moviegalaxies/408","moviegalaxies/409","moviegalaxies/410","moviegalaxies/411","moviegalaxies/412","moviegalaxies/413","moviegalaxies/414","moviegalaxies/415","moviegalaxies/416","moviegalaxies/417","moviegalaxies/418","moviegalaxies/419","moviegalaxies/420","moviegalaxies/421","moviegalaxies/422","moviegalaxies/423","moviegalaxies/424","moviegalaxies/426","moviegalaxies/427","moviegalaxies/429","moviegalaxies/430","moviegalaxies/432","moviegalaxies/433","moviegalaxies/434","moviegalaxies/435","moviegalaxies/436","moviegalaxies/437","moviegalaxies/438","moviegalaxies/439","moviegalaxies/442","moviegalaxies/443","moviegalaxies/444","moviegalaxies/445","moviegalaxies/446","moviegalaxies/447","moviegalaxies/449","moviegalaxies/450","moviegalaxies/451","moviegalaxies/452","moviegalaxies/453","moviegalaxies/454","moviegalaxies/455","moviegalaxies/456","moviegalaxies/457","moviegalaxies/458","moviegalaxies/459","moviegalaxies/460","moviegalaxies/461","moviegalaxies/462","moviegalaxies/464","moviegalaxies/465","moviegalaxies/466","moviegalaxies/467","moviegalaxies/468","moviegalaxies/469","moviegalaxies/470","moviegalaxies/471","moviegalaxies/472","moviegalaxies/474","moviegalaxies/475","moviegalaxies/476","moviegalaxies/477","moviegalaxies/478","moviegalaxies/481","moviegalaxies/482","moviegalaxies/483","moviegalaxies/484","moviegalaxies/485","moviegalaxies/486","moviegalaxies/487","moviegalaxies/488","moviegalaxies/489","moviegalaxies/490","moviegalaxies/491","moviegalaxies/494","moviegalaxies/495","moviegalaxies/496","moviegalaxies/497","moviegalaxies/498","moviegalaxies/499","moviegalaxies/500","moviegalaxies/501","moviegalaxies/503","moviegalaxies/505","moviegalaxies/506","moviegalaxies/507","moviegalaxies/508","moviegalaxies/509","moviegalaxies/510","moviegalaxies/511","moviegalaxies/512","moviegalaxies/513","moviegalaxies/515","moviegalaxies/516","moviegalaxies/517","moviegalaxies/518","moviegalaxies/519","moviegalaxies/520","moviegalaxies/521","moviegalaxies/523","moviegalaxies/524","moviegalaxies/525","moviegalaxies/526","moviegalaxies/527","moviegalaxies/528","moviegalaxies/529","moviegalaxies/530","moviegalaxies/531","moviegalaxies/532","moviegalaxies/533","moviegalaxies/534","moviegalaxies/535","moviegalaxies/536","moviegalaxies/537","moviegalaxies/538","moviegalaxies/539","moviegalaxies/541","moviegalaxies/542","moviegalaxies/543","moviegalaxies/544","moviegalaxies/545","moviegalaxies/547","moviegalaxies/548","moviegalaxies/549","moviegalaxies/550","moviegalaxies/551","moviegalaxies/552","moviegalaxies/553","moviegalaxies/555","moviegalaxies/556","moviegalaxies/558","moviegalaxies/559","moviegalaxies/560","moviegalaxies/562","moviegalaxies/564","moviegalaxies/565","moviegalaxies/566","moviegalaxies/567","moviegalaxies/569","moviegalaxies/570","moviegalaxies/571","moviegalaxies/572","moviegalaxies/573","moviegalaxies/575","moviegalaxies/577","moviegalaxies/578","moviegalaxies/579","moviegalaxies/580","moviegalaxies/581","moviegalaxies/582","moviegalaxies/583","moviegalaxies/584","moviegalaxies/586","moviegalaxies/587","moviegalaxies/591","moviegalaxies/592","moviegalaxies/593","moviegalaxies/594","moviegalaxies/595","moviegalaxies/599","moviegalaxies/600","moviegalaxies/601","moviegalaxies/602","moviegalaxies/603","moviegalaxies/605","moviegalaxies/606","moviegalaxies/607","moviegalaxies/608","moviegalaxies/609","moviegalaxies/610","moviegalaxies/611","moviegalaxies/615","moviegalaxies/616","moviegalaxies/618","moviegalaxies/619","moviegalaxies/620","moviegalaxies/621","moviegalaxies/622","moviegalaxies/623","moviegalaxies/624","moviegalaxies/625","moviegalaxies/628","moviegalaxies/629","moviegalaxies/630","moviegalaxies/631","moviegalaxies/632","moviegalaxies/633","moviegalaxies/634","moviegalaxies/635","moviegalaxies/636","moviegalaxies/637","moviegalaxies/638","moviegalaxies/640","moviegalaxies/641","moviegalaxies/642","moviegalaxies/643","moviegalaxies/644","moviegalaxies/645","moviegalaxies/647","moviegalaxies/648","moviegalaxies/649","moviegalaxies/650","moviegalaxies/651","moviegalaxies/652","moviegalaxies/653","moviegalaxies/655","moviegalaxies/656","moviegalaxies/657","moviegalaxies/658","moviegalaxies/659","moviegalaxies/660","moviegalaxies/661","moviegalaxies/662","moviegalaxies/664","moviegalaxies/665","moviegalaxies/666","moviegalaxies/667","moviegalaxies/668","moviegalaxies/669","moviegalaxies/670","moviegalaxies/671","moviegalaxies/672","moviegalaxies/673","moviegalaxies/675","moviegalaxies/677","moviegalaxies/678","moviegalaxies/679","moviegalaxies/682","moviegalaxies/683","moviegalaxies/685","moviegalaxies/686","moviegalaxies/687","moviegalaxies/688","moviegalaxies/689","moviegalaxies/691","moviegalaxies/692","moviegalaxies/693","moviegalaxies/694","moviegalaxies/695","moviegalaxies/696","moviegalaxies/697","moviegalaxies/698","moviegalaxies/699","moviegalaxies/700","moviegalaxies/701","moviegalaxies/703","moviegalaxies/704","moviegalaxies/705","moviegalaxies/706","moviegalaxies/707","moviegalaxies/708","moviegalaxies/710","moviegalaxies/712","moviegalaxies/713","moviegalaxies/714","moviegalaxies/715","moviegalaxies/716","moviegalaxies/717","moviegalaxies/718","moviegalaxies/719","moviegalaxies/720","moviegalaxies/721","moviegalaxies/723","moviegalaxies/724","moviegalaxies/725","moviegalaxies/727","moviegalaxies/728","moviegalaxies/729","moviegalaxies/730","moviegalaxies/731","moviegalaxies/732","moviegalaxies/733","moviegalaxies/734","moviegalaxies/735","moviegalaxies/737","moviegalaxies/738","moviegalaxies/739","moviegalaxies/740","moviegalaxies/741","moviegalaxies/742","moviegalaxies/743","moviegalaxies/744","moviegalaxies/745","moviegalaxies/746","moviegalaxies/747","moviegalaxies/748","moviegalaxies/749","moviegalaxies/750","moviegalaxies/751","moviegalaxies/753","moviegalaxies/754","moviegalaxies/755","moviegalaxies/756","moviegalaxies/757","moviegalaxies/758","moviegalaxies/760","moviegalaxies/761","moviegalaxies/762","moviegalaxies/764","moviegalaxies/765","moviegalaxies/766","moviegalaxies/767","moviegalaxies/768","moviegalaxies/769","moviegalaxies/770","moviegalaxies/771","moviegalaxies/772","moviegalaxies/773","moviegalaxies/774","moviegalaxies/775","moviegalaxies/777","moviegalaxies/779","moviegalaxies/780","moviegalaxies/781","moviegalaxies/782","moviegalaxies/783","moviegalaxies/784","moviegalaxies/785","moviegalaxies/786","moviegalaxies/787","moviegalaxies/788","moviegalaxies/790","moviegalaxies/791","moviegalaxies/792","moviegalaxies/793","moviegalaxies/794","moviegalaxies/797","moviegalaxies/798","moviegalaxies/799","moviegalaxies/800","moviegalaxies/801","moviegalaxies/802","moviegalaxies/803","moviegalaxies/804","moviegalaxies/805","moviegalaxies/806","moviegalaxies/808","moviegalaxies/809","moviegalaxies/810","moviegalaxies/811","moviegalaxies/812","moviegalaxies/814","moviegalaxies/815","moviegalaxies/816","moviegalaxies/817","moviegalaxies/818","moviegalaxies/819","moviegalaxies/820","moviegalaxies/821","moviegalaxies/822","moviegalaxies/824","moviegalaxies/825","moviegalaxies/826","moviegalaxies/827","moviegalaxies/828","moviegalaxies/829","moviegalaxies/830","moviegalaxies/831","moviegalaxies/832","moviegalaxies/833","moviegalaxies/834","moviegalaxies/835","moviegalaxies/836","moviegalaxies/837","moviegalaxies/839","moviegalaxies/840","moviegalaxies/841","moviegalaxies/843","moviegalaxies/844","moviegalaxies/846","moviegalaxies/847","moviegalaxies/848","moviegalaxies/849","moviegalaxies/850","moviegalaxies/851","moviegalaxies/852","moviegalaxies/853","moviegalaxies/854","moviegalaxies/855","moviegalaxies/857","moviegalaxies/858","moviegalaxies/859","moviegalaxies/860","moviegalaxies/861","moviegalaxies/862","moviegalaxies/863","moviegalaxies/865","moviegalaxies/867","moviegalaxies/869","moviegalaxies/870","moviegalaxies/871","moviegalaxies/872","moviegalaxies/873","moviegalaxies/874","moviegalaxies/875","moviegalaxies/876","moviegalaxies/877","moviegalaxies/878","moviegalaxies/879","moviegalaxies/880","moviegalaxies/881","moviegalaxies/883","moviegalaxies/884","moviegalaxies/885","moviegalaxies/887","moviegalaxies/888","moviegalaxies/889","moviegalaxies/891","moviegalaxies/892","moviegalaxies/893","moviegalaxies/895","moviegalaxies/896","moviegalaxies/899","moviegalaxies/901","moviegalaxies/902","moviegalaxies/904","moviegalaxies/905","moviegalaxies/906","moviegalaxies/907","moviegalaxies/908","moviegalaxies/910","moviegalaxies/911","moviegalaxies/912","moviegalaxies/913","moviegalaxies/914",
             "foodweb_baywet",
             "plant_pol_vazquez/Llao Llao","plant_pol_vazquez/Cerro Lopez","plant_pol_vazquez/Safariland","plant_pol_vazquez/Arroyo Goye","plant_pol_vazquez/Mascardi (nc)","plant_pol_vazquez/Mascardi (c)","plant_pol_vazquez/Quetrihue (nc)","plant_pol_vazquez/Quetrihue (c)","plant_pol_vazquez/All sites pooled",
             "dom/Adcock_2015a","dom/Adcock_2015b","dom/Alados_1992a","dom/Alados_1992b","dom/Alados_1992c","dom/Alados_1992d","dom/Alados_1992e","dom/Allee_1954","dom/Anderson_2016a","dom/Anderson_2016d","dom/Appleby_1983","dom/Archie_2006a","dom/Archie_2006b","dom/Archie_2006c","dom/Archie_2006d","dom/Archie_2006e","dom/Archie_2006f","dom/Archie_2006g","dom/Archie_2006h","dom/Archie_2006i","dom/Arlet_2015f","dom/Barette_1986","dom/Bennett_1939","dom/BergstromFedigan_2010b","dom/BergstromFedigan_2010c","dom/Berman_2004a","dom/Berman_2004b","dom/Berman_2004c","dom/Blatrix_2004b","dom/Blatrix_2004c","dom/Blatrix_2004d","dom/Blatrix_2004e","dom/Bonanni_2007a","dom/Bonanni_2007b","dom/Bonanni_2007c","dom/Bonanni_2017a","dom/Bonanni_2017b","dom/Bonanni_2017c","dom/Bonanni_2017d","dom/Bromley_1991","dom/Cafazzo_2010a","dom/Cafazzo_2010b","dom/Cheney_1977","dom/Cheney_1977b","dom/CluttonBrock_1976","dom/Collias_1950a","dom/Collias_1950b","dom/Collias_1950c","dom/Collias_1951a","dom/Collias_1951b","dom/Collias_1951c","dom/Correa_2013a","dom/Correa_2013d","dom/Cote_2000","dom/Cote_2000b","dom/Cote_2000c","dom/Cote_2000d","dom/Cui_2014","dom/David_2003a","dom/David_2003b","dom/DeLaFuente_2019","dom/DeLaO_2019a","dom/DeLaO_2019b","dom/DeLaO_2019c","dom/DeLaO_2019d","dom/deVries_2006","dom/deWaal_1977a","dom/deWaal_1977b","dom/deWaal_1985","dom/Diniz_2019","dom/Douglis_1948a","dom/Douglis_1948b","dom/Dubosq2013a","dom/Dubosq2013b","dom/Dubosq2013c","dom/Dubosq2013d","dom/Dubosq2013e","dom/Dubosq2013f","dom/Ellard_1989","dom/Essler_2016a","dom/Essler_2016c","dom/Essler_2016d","dom/Fairbanks_1994a","dom/Fairbanks_1994b","dom/Fairbanks_1994c","dom/Fairbanks_1994d","dom/Fairbanks_1994e","dom/Farentinos_1972a","dom/Farentinos_1972b","dom/Farentinos_1972c","dom/Farentinos_1972d","dom/Farentinos_1972e","dom/Farentinos_1972f","dom/Foerster_2016a","dom/Foerster_2016b","dom/Fournier_1995","dom/Fournier_1995b","dom/Frank_1986a","dom/Frank_1986b","dom/Franz_2015a","dom/Franz_2015b","dom/Franz_2015c","dom/Franz_2015d","dom/Franz_2015e","dom/Funkhouser_2018a","dom/Funkhouser_2018b","dom/Harcourt_1979","dom/Harcourt_1989","dom/Hartzler_1970","dom/Hass_1991","dom/Hass_1991b","dom/Hass_1991c","dom/Hausfater_1975a","dom/Hausfater_1975b","dom/Hausfater_1975c","dom/Hausfater_1982a","dom/Hausfater_1982b","dom/Hayaki_1989a","dom/Hayaki_1989b","dom/Heitor_2006","dom/HeitorVicente_2010","dom/Hewitt_2009a","dom/Hewitt_2009b","dom/Hewitt_2009c","dom/Hewitt_2009d","dom/Hirotani_1994","dom/Hirsch_2007a","dom/Hirsch_2007b","dom/Hirsch_2007c","dom/Hirsch_2007d","dom/Hobson_2015a","dom/Hobson_2015b","dom/Holekamp_1991","dom/Holekamp_1993a","dom/Holekamp_1993b","dom/Isbell_1998a","dom/Isbell_1998b","dom/Ito_1993a","dom/Ito_1993b","dom/Ito_1993c","dom/Izar_2006","dom/IzawaWatanabe_2011","dom/Jenks_1995a","dom/Jenks_1995b","dom/Johnson_2017","dom/Jones_1980a","dom/Jones_1980b","dom/Kaufmann_1974","dom/Kikkawa_1980","dom/Koenig_2004","dom/Kohda_1991","dom/Kolodziejczyk_2005","dom/Korstjens_2002","dom/Koutnik_1981","dom/Koutnik_1981b","dom/Kurvers_2009","dom/Lahti_1994a","dom/Lahti_1994b","dom/Langbein_2004a","dom/Langbein_2004b","dom/Langbein_2004c","dom/Langbein_2004d","dom/Langley_2018","dom/Lee_1979a","dom/Lee_1979b","dom/Lee_2017a","dom/Lee_2017b","dom/Lee_2017c","dom/Lee_2017d","dom/Lee_2017e","dom/Lee_2017f","dom/Lee_2018a","dom/Lee_2018b","dom/Lee_2018c","dom/Lee_2018d","dom/Lee_2018e","dom/Lee_2019a","dom/Lee_2019b","dom/Lee_2019c","dom/Lee_2019d","dom/Lee_2019e","dom/Lott_1979","dom/Lott_1987","dom/Lu_2008a","dom/Lu_2008b","dom/Malherbe_2007","dom/Marler_1955","dom/Masure_1934","dom/Masure_1934a","dom/Masure_1934b","dom/Mather_1985","dom/Matsuda_2012","dom/McCune_2019a","dom/McCune_2019c","dom/McCune_2019d","dom/McCune_2019e","dom/McCune_2019f","dom/McDougall_2010","dom/McMahan_1984","dom/Miller_2010a","dom/Miller_2010b","dom/Miller_2010c","dom/Moller_1987a","dom/Moller_1987b","dom/Moller_1987c","dom/Monnin_1999","dom/Moore_1978a","dom/Moore_1978b","dom/Murray_2007","dom/Mwamende_2009a","dom/Mwamende_2009b","dom/Myrberg_1972","dom/Myrberg_1974","dom/Nakano_1994","dom/Nakano_1995","dom/Natoli_1991","dom/Nelissen_1985a","dom/Nelissen_1985b","dom/NewtonFisher_2004a","dom/NewtonFisher_2004b","dom/Norscia_2015a","dom/Norscia_2015b","dom/Norscia_2015c","dom/Norscia_2015d","dom/Norscia_2015e","dom/Norscia_2015f","dom/Norscia_2015g","dom/Norscia_2015h","dom/Norscia_2015i","dom/Norscia_2015j","dom/Ortius_1995","dom/Oshea_1976","dom/Owens_1996","dom/Painter_2018","dom/Paoli_2006","dom/Pardi_1948a","dom/Pardi_1948b","dom/Parsons_1980a","dom/Parsons_1980b","dom/Patterson_1977","dom/Payne_2003","dom/Poisbleau_2005a","dom/Poisbleau_2005b","dom/Poisbleau_2005c","dom/Poisbleau_2006a","dom/Poisbleau_2006b","dom/Post_1992","dom/Prieto_1978","dom/Reason_1988","dom/Richter_2009","dom/RizaldiWatanabe_2010a","dom/RizaldiWatanabe_2010b","dom/Robbins_2008","dom/Roell_1978","dom/Rossler_2017","dom/Rovero_1999a","dom/Rovero_1999b","dom/Russell_1970a","dom/Russell_1970b","dom/Rutberg_1986a","dom/Rutberg_1986b","dom/Samuels_1987","dom/Samuels_1987b","dom/Sandel_2017a","dom/Sandel_2017b","dom/SatohOhkawara_2008a","dom/SatohOhkawara_2008b","dom/SatohOhkawara_2008c","dom/ScheinFohrman_1955","dom/SchjelderupEbbe_1922a","dom/SchjelderupEbbe_1922b","dom/ScottLockhard_1999a","dom/ScottLockhard_1999b","dom/Seibert_2001","dom/Setchell_2005a","dom/Setchell_2005b","dom/SetiaVanSchaik_2007","dom/Seyfarth_1976a","dom/Seyfarth_1976b","dom/Seyfarth_1976c","dom/Seyfarth_1976d","dom/Seyfarth_1976e","dom/Seyfarth_1976f","dom/Sharpe_2013a","dom/Sharpe_2013b","dom/Sharpe_2013c","dom/Sharpe_2013d","dom/Shimoji_2014a","dom/Shimoji_2014b","dom/Shimoji_2014c","dom/Shoemaker_1939","dom/Silk_2019a","dom/Slotow_1993","dom/Smith_1976","dom/Smith_2011a","dom/Solberg_1997a","dom/Solberg_1997b","dom/Solberg_1997c","dom/SomersNel_1998a","dom/SomersNel_1998b","dom/SomersNel_1998c","dom/Stamps_1978","dom/Sterck_1997","dom/Strauss_2019a","dom/Strauss_2019b","dom/Strauss_2019c","dom/Strauss_2019d","dom/StrayerCummins_1980","dom/StrayerCummins_1980b","dom/StrayerCummins_1980c","dom/StrayerCummins_1980d","dom/Struhsaker_1967a","dom/Struhsaker_1967b","dom/Sullivan_1982","dom/Surbeck_2011","dom/SurbeckHohmann_2013b","dom/Tamm_1977","dom/Tamura_1988","dom/Tarvin_1997","dom/Thompson_1960a","dom/Thompson_1960b","dom/Thompson_1960c","dom/Thompson_1960d","dom/Thompson_1960e","dom/Thompson_1993a","dom/Thompson_1993b","dom/Tilson_1984","dom/Tong_2020a","dom/Tong_2020b","dom/Tong_2020c","dom/Tong_2020d","dom/Tong_2020e","dom/Tong_2020f","dom/Tong_2020g","dom/Tong_2020h","dom/Tong_2020i","dom/Tong_2020j","dom/Torr_1996a","dom/Torr_1996b","dom/Torr_1996c","dom/Trebouet_2019a","dom/Trebouet_2019b","dom/Trebouet_2019c","dom/Trebouet_2019d","dom/Trunzer_1999","dom/Utt_2008a","dom/ValLaillet_2008a","dom/ValLaillet_2008b","dom/ValLaillet_2008c","dom/ValLaillet_2008d","dom/ValLaillet_2008e","dom/ValLaillet_2008f","dom/VanDessel_2018a","dom/VanDessel_2018b","dom/VanDierendonck_1995","dom/VanHooffWensing_1987","dom/Varley_1966","dom/Varley_1966b","dom/Vervaecke_2000","dom/Vervaecke_2010a","dom/Vervaecke_2010b","dom/Vervaecke_2010c","dom/Vervaecke_2010d","dom/Vilette_2020","dom/Waterhouse_1976a","dom/Waterhouse_1976b","dom/Waterhouse_1976c","dom/Watson_1970a","dom/Watson_1970b","dom/Watt_1986a","dom/Watt_1986b","dom/Watt_1986c","dom/Watt_1986d","dom/Watt_1986e","dom/Watt_1986f","dom/Watts_1985","dom/Watts_1994a","dom/Watts_1994b","dom/Watts_1994c","dom/Wells_1979","dom/WestEberhard_1986","dom/White_2007a","dom/White_2007b","dom/White_2007c","dom/Wikberg_2013a","dom/Wikberg_2013b","dom/Wikberg_2013c","dom/Wikberg_2013d","dom/Wikberg_2013e","dom/Wikberg_2013f","dom/Williams_1972","dom/Williamson_2016a","dom/Williamson_2016b","dom/Williamson_2016c","dom/Williamson_2016d","dom/Williamson_2016e","dom/Williamson_2016f","dom/Williamson_2016g","dom/Williamson_2016h","dom/Williamson_2016i","dom/Williamson_2016j","dom/Williamson_2016k","dom/Williamson_2017a","dom/Williamson_2017b","dom/Williamson_2017c","dom/Williamson_2017d","dom/Williamson_2017e","dom/Williamson_2017f","dom/Williamson_2017g","dom/Williamson_2017h","dom/Williamson_2017i","dom/Williamson_2017j","dom/Williamson_2017k","dom/Williamson_2017l","dom/Williamson_2017m","dom/Williamson_2017n","dom/Williamson_2017o","dom/Williamson_2019a","dom/Williamson_2019b","dom/Williamson_2019c","dom/Williamson_2019d","dom/Williamson_2019e","dom/Williamson_2019f","dom/Williamson_2019g","dom/Williamson_2019h","dom/Wittemeyer_2007","dom/Wittig_2003","dom/Yasukawa_1983a","dom/Yasukawa_1983b","dom/Yasukawa_1983c","dom/Yasukawa_1983d","dom/Yasukawa_1983e","dom/Yasukawa_1983f","dom/Yasukawa_1983g","dom/Yasukawa_1983h","dom/Zine_2000",
             "cintestinalis",
             "faculty_hiring/computer_science","faculty_hiring/business","faculty_hiring/history",
             "fao_trade","residence_hall","un_migrations","sp_primary_school","celegansneural",
             "sp_high_school/proximity","sp_high_school/diaries","sp_high_school/survey","sp_high_school/facebook",
             "london_transport",
             "sp_colocation/Thiers13","sp_colocation/SFHH","sp_colocation/LyonSchool","sp_colocation/LH10","sp_colocation/InVS15","sp_colocation/InVS13",
             "webkb/webkb_wisconsin_link1","webkb/webkb_wisconsin_cocite","webkb/webkb_washington_link1","webkb/webkb_washington_cocite","webkb/webkb_cornell_link1","webkb/webkb_cornell_cocite","webkb/webkb_texas_link1","webkb/webkb_texas_cocite",
            "celegans_2019/male_chemical","celegans_2019/male_gap_junction","celegans_2019/hermaphrodite_chemical","celegans_2019/hermaphrodite_gap_junction","celegans_2019/male_chemical_synapse","celegans_2019/male_gap_junction_synapse","celegans_2019/hermaphrodite_chemical_synapse","celegans_2019/hermaphrodite_gap_junction_synapse","celegans_2019/male_chemical_corrected","celegans_2019/male_gap_junction_corrected","celegans_2019/hermaphrodite_chemical_corrected","celegans_2019/hermaphrodite_gap_junction_corrected",
            #"messal_shale", #DOES NOT WORK
             "plant_pol_kato",
             "copenhagen/sms","copenhagen/fb_friends","copenhagen/calls","copenhagen/bt",
            "product_space/HS","product_space/SITC",
            "unicodelang",
            "budapest_connectome/all_20k","budapest_connectome/all_200k","budapest_connectome/all_1m","budapest_connectome/female_20k","budapest_connectome/female_200k","budapest_connectome/female_1m","budapest_connectome/male_20k","budapest_connectome/male_200k","budapest_connectome/male_1m",
            "new_zealand_collab","netscience","bible_nouns",
            # "add_health/comm1","add_health/comm2","add_health/comm3","add_health/comm4",#there are about 100 graphs. we consider only 4. BUT IT DOES NOT WORK: files offline
            "openflights","bitcoin_alpha","bitcoin_trust","advogato","foldoc",
            "physics_collab/pierreAuger","physics_collab/arXiv",
            "escorts","fly_hemibrain",
            "arxiv_collab/cond-mat-1999","arxiv_collab/cond-mat-2003","arxiv_collab/cond-mat-2005","arxiv_collab/astro-ph-1999","arxiv_collab/hep-th-1999",
            "us_agencies/alabama","us_agencies/alaska","us_agencies/arizona","us_agencies/arkansas","us_agencies/california","us_agencies/colorado","us_agencies/connecticut","us_agencies/delaware","us_agencies/florida","us_agencies/georgia","us_agencies/hawaii","us_agencies/idaho","us_agencies/illinois","us_agencies/indiana","us_agencies/iowa","us_agencies/kansas","us_agencies/kentucky","us_agencies/louisiana","us_agencies/maine","us_agencies/maryland","us_agencies/massachusetts","us_agencies/michigan","us_agencies/minnesota","us_agencies/mississippi","us_agencies/missouri","us_agencies/montana","us_agencies/nebraska","us_agencies/nevada","us_agencies/newhampshire","us_agencies/newjersey","us_agencies/newmexico","us_agencies/newyork","us_agencies/northcarolina","us_agencies/northdakota","us_agencies/ohio","us_agencies/oklahoma","us_agencies/oregon","us_agencies/pennsylvania","us_agencies/rhodeisland","us_agencies/southcarolina","us_agencies/southdakota","us_agencies/tennessee","us_agencies/texas","us_agencies/utah","us_agencies/vermont","us_agencies/virginia","us_agencies/washington","us_agencies/westvirginia","us_agencies/wisconsin","us_agencies/wyoming","us_agencies/aggregate",
           "eu_procurements_alt/AT_2008","eu_procurements_alt/AT_2009","eu_procurements_alt/AT_2010","eu_procurements_alt/AT_2011","eu_procurements_alt/AT_2012","eu_procurements_alt/AT_2013","eu_procurements_alt/AT_2014","eu_procurements_alt/AT_2015","eu_procurements_alt/AT_2016","eu_procurements_alt/BE_2008","eu_procurements_alt/BE_2009","eu_procurements_alt/BE_2010","eu_procurements_alt/BE_2011","eu_procurements_alt/BE_2012","eu_procurements_alt/BE_2013","eu_procurements_alt/BE_2014","eu_procurements_alt/BE_2015","eu_procurements_alt/BE_2016","eu_procurements_alt/BG_2008","eu_procurements_alt/BG_2009","eu_procurements_alt/BG_2010","eu_procurements_alt/BG_2011","eu_procurements_alt/BG_2012","eu_procurements_alt/BG_2013","eu_procurements_alt/BG_2014","eu_procurements_alt/BG_2015","eu_procurements_alt/BG_2016","eu_procurements_alt/CY_2008","eu_procurements_alt/CY_2009","eu_procurements_alt/CY_2010","eu_procurements_alt/CY_2011","eu_procurements_alt/CY_2012","eu_procurements_alt/CY_2013","eu_procurements_alt/CY_2014","eu_procurements_alt/CY_2015","eu_procurements_alt/CY_2016","eu_procurements_alt/CZ_2008","eu_procurements_alt/CZ_2009","eu_procurements_alt/CZ_2010","eu_procurements_alt/CZ_2011","eu_procurements_alt/CZ_2012","eu_procurements_alt/CZ_2013","eu_procurements_alt/CZ_2014","eu_procurements_alt/CZ_2015","eu_procurements_alt/CZ_2016","eu_procurements_alt/DE_2008","eu_procurements_alt/DE_2009","eu_procurements_alt/DE_2010","eu_procurements_alt/DE_2011","eu_procurements_alt/DE_2012","eu_procurements_alt/DE_2013","eu_procurements_alt/DE_2014","eu_procurements_alt/DE_2015","eu_procurements_alt/DE_2016","eu_procurements_alt/DK_2008","eu_procurements_alt/DK_2009","eu_procurements_alt/DK_2010","eu_procurements_alt/DK_2011","eu_procurements_alt/DK_2012","eu_procurements_alt/DK_2013","eu_procurements_alt/DK_2014","eu_procurements_alt/DK_2015","eu_procurements_alt/DK_2016","eu_procurements_alt/EE_2008","eu_procurements_alt/EE_2009","eu_procurements_alt/EE_2010","eu_procurements_alt/EE_2011","eu_procurements_alt/EE_2012","eu_procurements_alt/EE_2013","eu_procurements_alt/EE_2014","eu_procurements_alt/EE_2015","eu_procurements_alt/EE_2016","eu_procurements_alt/ES_2008","eu_procurements_alt/ES_2009","eu_procurements_alt/ES_2010","eu_procurements_alt/ES_2011","eu_procurements_alt/ES_2012","eu_procurements_alt/ES_2013","eu_procurements_alt/ES_2014","eu_procurements_alt/ES_2015","eu_procurements_alt/ES_2016","eu_procurements_alt/FI_2008","eu_procurements_alt/FI_2009","eu_procurements_alt/FI_2010","eu_procurements_alt/FI_2011","eu_procurements_alt/FI_2012","eu_procurements_alt/FI_2013","eu_procurements_alt/FI_2014","eu_procurements_alt/FI_2015","eu_procurements_alt/FI_2016","eu_procurements_alt/FR_2008","eu_procurements_alt/FR_2009","eu_procurements_alt/FR_2010","eu_procurements_alt/FR_2011","eu_procurements_alt/FR_2012","eu_procurements_alt/FR_2013","eu_procurements_alt/FR_2014","eu_procurements_alt/FR_2015","eu_procurements_alt/FR_2016","eu_procurements_alt/GR_2008","eu_procurements_alt/GR_2009","eu_procurements_alt/GR_2010","eu_procurements_alt/GR_2011","eu_procurements_alt/GR_2012","eu_procurements_alt/GR_2013","eu_procurements_alt/GR_2014","eu_procurements_alt/GR_2015","eu_procurements_alt/GR_2016","eu_procurements_alt/HU_2008","eu_procurements_alt/HU_2009","eu_procurements_alt/HU_2010","eu_procurements_alt/HU_2011","eu_procurements_alt/HU_2012","eu_procurements_alt/HU_2013","eu_procurements_alt/HU_2014","eu_procurements_alt/HU_2015","eu_procurements_alt/HU_2016","eu_procurements_alt/IE_2008","eu_procurements_alt/IE_2009","eu_procurements_alt/IE_2010","eu_procurements_alt/IE_2011","eu_procurements_alt/IE_2012","eu_procurements_alt/IE_2013","eu_procurements_alt/IE_2014","eu_procurements_alt/IE_2015","eu_procurements_alt/IE_2016","eu_procurements_alt/IT_2008","eu_procurements_alt/IT_2009","eu_procurements_alt/IT_2010","eu_procurements_alt/IT_2011","eu_procurements_alt/IT_2012","eu_procurements_alt/IT_2013","eu_procurements_alt/IT_2014","eu_procurements_alt/IT_2015","eu_procurements_alt/IT_2016","eu_procurements_alt/LT_2008","eu_procurements_alt/LT_2009","eu_procurements_alt/LT_2010","eu_procurements_alt/LT_2011","eu_procurements_alt/LT_2012","eu_procurements_alt/LT_2013","eu_procurements_alt/LT_2014","eu_procurements_alt/LT_2015","eu_procurements_alt/LT_2016","eu_procurements_alt/LV_2008","eu_procurements_alt/LV_2009","eu_procurements_alt/LV_2010","eu_procurements_alt/LV_2011","eu_procurements_alt/LV_2012","eu_procurements_alt/LV_2013","eu_procurements_alt/LV_2014","eu_procurements_alt/LV_2015","eu_procurements_alt/LV_2016","eu_procurements_alt/NL_2008","eu_procurements_alt/NL_2009","eu_procurements_alt/NL_2010","eu_procurements_alt/NL_2011","eu_procurements_alt/NL_2012","eu_procurements_alt/NL_2013","eu_procurements_alt/NL_2014","eu_procurements_alt/NL_2015","eu_procurements_alt/NL_2016","eu_procurements_alt/NO_2008","eu_procurements_alt/NO_2009","eu_procurements_alt/NO_2010","eu_procurements_alt/NO_2011","eu_procurements_alt/NO_2012","eu_procurements_alt/NO_2013","eu_procurements_alt/NO_2014","eu_procurements_alt/NO_2015","eu_procurements_alt/NO_2016","eu_procurements_alt/PL_2008","eu_procurements_alt/PL_2009","eu_procurements_alt/PL_2010","eu_procurements_alt/PL_2011","eu_procurements_alt/PL_2012","eu_procurements_alt/PL_2013","eu_procurements_alt/PL_2014","eu_procurements_alt/PL_2015","eu_procurements_alt/PL_2016","eu_procurements_alt/PT_2008","eu_procurements_alt/PT_2009","eu_procurements_alt/PT_2010","eu_procurements_alt/PT_2011","eu_procurements_alt/PT_2012","eu_procurements_alt/PT_2013","eu_procurements_alt/PT_2014","eu_procurements_alt/PT_2015","eu_procurements_alt/PT_2016","eu_procurements_alt/RO_2008","eu_procurements_alt/RO_2009","eu_procurements_alt/RO_2010","eu_procurements_alt/RO_2011","eu_procurements_alt/RO_2012","eu_procurements_alt/RO_2013","eu_procurements_alt/RO_2014","eu_procurements_alt/RO_2015","eu_procurements_alt/RO_2016","eu_procurements_alt/SE_2008","eu_procurements_alt/SE_2009","eu_procurements_alt/SE_2010","eu_procurements_alt/SE_2011","eu_procurements_alt/SE_2012","eu_procurements_alt/SE_2013","eu_procurements_alt/SE_2014","eu_procurements_alt/SE_2015","eu_procurements_alt/SE_2016","eu_procurements_alt/SI_2008","eu_procurements_alt/SI_2009","eu_procurements_alt/SI_2010","eu_procurements_alt/SI_2011","eu_procurements_alt/SI_2012","eu_procurements_alt/SI_2013","eu_procurements_alt/SI_2014","eu_procurements_alt/SI_2015","eu_procurements_alt/SI_2016","eu_procurements_alt/SK_2008","eu_procurements_alt/SK_2009","eu_procurements_alt/SK_2010","eu_procurements_alt/SK_2011","eu_procurements_alt/SK_2012","eu_procurements_alt/SK_2013","eu_procurements_alt/SK_2014","eu_procurements_alt/SK_2015","eu_procurements_alt/SK_2016","eu_procurements_alt/UK_2008","eu_procurements_alt/UK_2009","eu_procurements_alt/UK_2010","eu_procurements_alt/UK_2011","eu_procurements_alt/UK_2012","eu_procurements_alt/UK_2013","eu_procurements_alt/UK_2014","eu_procurements_alt/UK_2015","eu_procurements_alt/UK_2016",
            "jester","libimseti","wiki_article_words","bookcrossing",
            "twitter_higgs/social","twitter_higgs/retweet","twitter_higgs/reply","twitter_higgs/mention","twitter_higgs/activity",
            "eu_procurements"  ,"epinions",
             "mag_history_coauthor/max-25-bip","mag_history_coauthor/max-25-proj","mag_history_coauthor/full-bip","mag_history_coauthor/full-proj",
             "mag_geology_coauthor/max-25-bip","mag_geology_coauthor/max-25-proj","mag_geology_coauthor/full-bip","mag_geology_coauthor/full-proj",
             "amazon_ratings",
             "dblp_simplices/max-25-bip","dblp_simplices/max-25-proj","dblp_simplices/full-bip","dblp_simplices/full-proj",
             "bag_of_words/enron","bag_of_words/kos","bag_of_words/nips","bag_of_words/nytimes", 
             "microsoft_concept"
             ,"bag_of_words/pubmed" #, TOO BIG    
             ,"yahoo_song" #DOES NOT WORK ON COLAB: TOO MUCH RAM
]

#Unfortunately, not all models classified as weighted have weights. We handle these cases setting deafult weight 1
#model_names=["hens"] # DOES NOT HAVE WEIGHTS! The default '1' will be set to all edges

prefix_to_weight={'eu_procurements_alt/':'pctSingleBid','arxiv_collab/':'value','bag_of_words/':'count',
                  'budapest_connectome/':'occurences','celegans_2019':'connectivity',
                  'us_agencies/':'link_counts','plant_pol_vazquez':'count','microsoft_concept':'count',
                  'dblp_simplices/':'time'}

#Set flags on where to store the obtained CSV files
download_file=False
store_on_gdrive=True
path_g_drive="measureGraphs/"
measure_allgraphs(model_names,download_file,store_on_gdrive,path_g_drive,prefix_to_weight,"weighted")

SyntaxError: non-default argument follows default argument (2314955081.py, line 12)

## Unweighted

In [ ]:
#All unweighted networks on 21/07/2022 from smallest number of nodes to largest#
# Taken from https://networks.skewed.de/?tags=Unweighted after clicking on 'Nodes'
model_names=[
    'sa_companies',
    #"florentine_families", does not work
    'high_tech_company','moreno_taro','november17',
    'sp_baboons/observational','sp_baboons/sensor',
    'dutch_school/klas12b-net-1','dutch_school/klas12b-net-2','dutch_school/klas12b-net-3','dutch_school/klas12b-net-3m','dutch_school/klas12b-net-4','dutch_school/klas12b-net-4m','dutch_school/klas12b-primary',
    'zebras','7th_graders','karate/77','karate/78','dutch_criticism','montreal','ceo_club','elite',
    'macaque_neural','sp_kenyan_households','contiguous_usa','cs_department','dolphins','terrorists_911',
    'law_firm','sp_hospital','reality_mining','swingers','polbooks','adjnoun',
    'fresh_webs/AkatoreA','fresh_webs/AkatoreB','fresh_webs/Berwick','fresh_webs/Blackrock','fresh_webs/Broad','fresh_webs/Canton','fresh_webs/Catlins','fresh_webs/Coweeta1','fresh_webs/Coweeta17','fresh_webs/DempstersAu','fresh_webs/DempstersSp','fresh_webs/DempstersSu','fresh_webs/German','fresh_webs/Healy','fresh_webs/Kyeburn','fresh_webs/LilKyeburn','fresh_webs/Martins','fresh_webs/Narrowdale','fresh_webs/NorthCol','fresh_webs/Powder','fresh_webs/Stony','fresh_webs/SuttonAu','fresh_webs/SuttonSp','fresh_webs/SuttonSu','fresh_webs/Troy','fresh_webs/Venlaw',
    'sp_hypertext/contacts','sp_hypertext/intervals',
    'football','football_tsevans','revolution','email_company',
    'sp_high_school_new/2011','sp_high_school_new/2012',
    'foodweb_little_rock','student_cooperation','jazz_collab','interactome_pdz','physician_trust','contact',
    'malaria_genes/HVR_1','malaria_genes/HVR_2','malaria_genes/HVR_3','malaria_genes/HVR_4','malaria_genes/HVR_5','malaria_genes/HVR_6','malaria_genes/HVR_7','malaria_genes/HVR_8','malaria_genes/HVR_9','malaria_genes/combined',
    'sp_high_school/proximity','sp_high_school/diaries','sp_high_school/survey','sp_high_school/facebook',
    'marvel_partnerships','kidnappings','facebook_friends',
    'ugandan_village/friendship-1','ugandan_village/friendship-2','ugandan_village/friendship-3','ugandan_village/friendship-4','ugandan_village/friendship-5','ugandan_village/friendship-6','ugandan_village/friendship-7','ugandan_village/friendship-8','ugandan_village/friendship-9','ugandan_village/friendship-10','ugandan_village/friendship-11','ugandan_village/friendship-12','ugandan_village/friendship-13','ugandan_village/friendship-14','ugandan_village/friendship-15','ugandan_village/friendship-16','ugandan_village/friendship-17','ugandan_village/health-advice_1','ugandan_village/health-advice_2','ugandan_village/health-advice_3','ugandan_village/health-advice_4','ugandan_village/health-advice_5','ugandan_village/health-advice_6','ugandan_village/health-advice_7','ugandan_village/health-advice_8','ugandan_village/health-advice_9','ugandan_village/health-advice_10','ugandan_village/health-advice_11','ugandan_village/health-advice_12','ugandan_village/health-advice_13','ugandan_village/health-advice_14','ugandan_village/health-advice_15','ugandan_village/health-advice_16','ugandan_village/health-advice_17',
    'sp_colocation/Thiers13','sp_colocation/SFHH','sp_colocation/LyonSchool','sp_colocation/LH10','sp_colocation/InVS15','sp_colocation/InVS13',
    'ecoli_transcription/v1.0','ecoli_transcription/v1.1',
    'eu_airlines','celegans_metabolic','wiki_science',
    'internet_top_pop/Aarnet','internet_top_pop/Abilene','internet_top_pop/Abvt','internet_top_pop/Aconet','internet_top_pop/Agis','internet_top_pop/Ai3','internet_top_pop/Airtel','internet_top_pop/Amres','internet_top_pop/Ans','internet_top_pop/Arn','internet_top_pop/Arnes','internet_top_pop/Arpanet196912','internet_top_pop/Arpanet19706','internet_top_pop/Arpanet19719','internet_top_pop/Arpanet19723','internet_top_pop/Arpanet19728','internet_top_pop/AsnetAm','internet_top_pop/Atmnet','internet_top_pop/AttMpls','internet_top_pop/Azrena','internet_top_pop/Bandcon','internet_top_pop/Basnet','internet_top_pop/Bbnplanet','internet_top_pop/Bellcanada','internet_top_pop/Bellsouth','internet_top_pop/Belnet2003','internet_top_pop/Belnet2004','internet_top_pop/Belnet2005','internet_top_pop/Belnet2006','internet_top_pop/Belnet2007','internet_top_pop/Belnet2008','internet_top_pop/Belnet2009','internet_top_pop/Belnet2010','internet_top_pop/BeyondTheNetwork','internet_top_pop/Bics','internet_top_pop/Biznet','internet_top_pop/Bren','internet_top_pop/BsonetEurope','internet_top_pop/BtAsiaPac','internet_top_pop/BtEurope','internet_top_pop/BtLatinAmerica','internet_top_pop/BtNorthAmerica','internet_top_pop/Canerie','internet_top_pop/Carnet','internet_top_pop/Cernet','internet_top_pop/Cesnet1993','internet_top_pop/Cesnet1997','internet_top_pop/Cesnet1999','internet_top_pop/Cesnet2001','internet_top_pop/Cesnet200304','internet_top_pop/Cesnet200511','internet_top_pop/Cesnet200603','internet_top_pop/Cesnet200706','internet_top_pop/Cesnet201006','internet_top_pop/Chinanet','internet_top_pop/Claranet','internet_top_pop/Cogentco','internet_top_pop/Colt','internet_top_pop/Columbus','internet_top_pop/Compuserve','internet_top_pop/CrlNetworkServices','internet_top_pop/Cudi','internet_top_pop/Cwix','internet_top_pop/Cynet','internet_top_pop/Darkstrand','internet_top_pop/Dataxchange','internet_top_pop/Deltacom','internet_top_pop/DeutscheTelekom','internet_top_pop/Dfn','internet_top_pop/DialtelecomCz','internet_top_pop/Digex','internet_top_pop/Easynet','internet_top_pop/Eenet','internet_top_pop/EliBackbone','internet_top_pop/Epoch','internet_top_pop/Ernet','internet_top_pop/Esnet','internet_top_pop/Eunetworks','internet_top_pop/Evolink','internet_top_pop/Fatman','internet_top_pop/Fccn','internet_top_pop/Forthnet','internet_top_pop/Funet','internet_top_pop/Gambia','internet_top_pop/Garr199901','internet_top_pop/Garr199904','internet_top_pop/Garr199905','internet_top_pop/Garr200109','internet_top_pop/Garr200112','internet_top_pop/Garr200212','internet_top_pop/Garr200404','internet_top_pop/Garr200902','internet_top_pop/Garr200908','internet_top_pop/Garr200909','internet_top_pop/Garr200912','internet_top_pop/Garr201001','internet_top_pop/Garr201003','internet_top_pop/Garr201004','internet_top_pop/Garr201005','internet_top_pop/Garr201007','internet_top_pop/Garr201008','internet_top_pop/Garr201010','internet_top_pop/Garr201012','internet_top_pop/Garr201101','internet_top_pop/Garr201102','internet_top_pop/Garr201103','internet_top_pop/Garr201104','internet_top_pop/Garr201105','internet_top_pop/Garr201107','internet_top_pop/Garr201108','internet_top_pop/Garr201109','internet_top_pop/Garr201110','internet_top_pop/Garr201111','internet_top_pop/Garr201112','internet_top_pop/Garr201201','internet_top_pop/Gblnet','internet_top_pop/Geant2001','internet_top_pop/Geant2009','internet_top_pop/Geant2010','internet_top_pop/Geant2012','internet_top_pop/Getnet','internet_top_pop/Globalcenter','internet_top_pop/Globenet','internet_top_pop/Goodnet','internet_top_pop/Grena','internet_top_pop/Gridnet','internet_top_pop/Grnet','internet_top_pop/GtsCe','internet_top_pop/GtsCzechRepublic','internet_top_pop/GtsHungary','internet_top_pop/GtsPoland','internet_top_pop/GtsRomania','internet_top_pop/GtsSlovakia','internet_top_pop/Harnet','internet_top_pop/Heanet','internet_top_pop/HiberniaCanada','internet_top_pop/HiberniaGlobal','internet_top_pop/HiberniaIreland','internet_top_pop/HiberniaNireland','internet_top_pop/HiberniaUk','internet_top_pop/HiberniaUs','internet_top_pop/Highwinds','internet_top_pop/HostwayInternational','internet_top_pop/HurricaneElectric','internet_top_pop/Ibm','internet_top_pop/Iij','internet_top_pop/Iinet','internet_top_pop/Ilan','internet_top_pop/Integra','internet_top_pop/Intellifiber','internet_top_pop/Internetmci','internet_top_pop/Internode','internet_top_pop/Interoute','internet_top_pop/Intranetwork','internet_top_pop/Ion','internet_top_pop/IowaStatewideFiberMap','internet_top_pop/Iris','internet_top_pop/Istar','internet_top_pop/Itnet','internet_top_pop/Janetbackbone','internet_top_pop/JanetExternal','internet_top_pop/Janetlense','internet_top_pop/Jgn2Plus','internet_top_pop/Karen','internet_top_pop/Kdl','internet_top_pop/KentmanApr2007','internet_top_pop/KentmanAug2005','internet_top_pop/KentmanFeb2008','internet_top_pop/KentmanJan2011','internet_top_pop/KentmanJul2005','internet_top_pop/Kreonet','internet_top_pop/LambdaNet','internet_top_pop/Latnet','internet_top_pop/Layer42','internet_top_pop/Litnet','internet_top_pop/Marnet','internet_top_pop/Marwan','internet_top_pop/Missouri','internet_top_pop/Mren','internet_top_pop/Myren','internet_top_pop/Napnet','internet_top_pop/Navigata','internet_top_pop/Netrail','internet_top_pop/NetworkUsa','internet_top_pop/Nextgen','internet_top_pop/Niif','internet_top_pop/Noel','internet_top_pop/Nordu1989','internet_top_pop/Nordu1997','internet_top_pop/Nordu2005','internet_top_pop/Nordu2010','internet_top_pop/Nsfcnet','internet_top_pop/Nsfnet','internet_top_pop/Ntelos','internet_top_pop/Ntt','internet_top_pop/Oteglobe','internet_top_pop/Oxford','internet_top_pop/Pacificwave','internet_top_pop/Packetexchange','internet_top_pop/Padi','internet_top_pop/Palmetto','internet_top_pop/Peer1','internet_top_pop/Pern','internet_top_pop/PionierL1','internet_top_pop/PionierL3','internet_top_pop/Psinet','internet_top_pop/Quest','internet_top_pop/RedBestel','internet_top_pop/Rediris','internet_top_pop/Renam','internet_top_pop/Renater1999','internet_top_pop/Renater2001','internet_top_pop/Renater2004','internet_top_pop/Renater2006','internet_top_pop/Renater2008','internet_top_pop/Renater2010','internet_top_pop/Restena','internet_top_pop/Reuna','internet_top_pop/Rhnet','internet_top_pop/Rnp','internet_top_pop/Roedunet','internet_top_pop/RoedunetFibre','internet_top_pop/Sago','internet_top_pop/Sanet','internet_top_pop/Sanren','internet_top_pop/Savvis','internet_top_pop/Shentel','internet_top_pop/Sinet','internet_top_pop/Singaren','internet_top_pop/Spiralight','internet_top_pop/Sprint','internet_top_pop/Sunet','internet_top_pop/Surfnet','internet_top_pop/Switch','internet_top_pop/SwitchL3','internet_top_pop/Syringa','internet_top_pop/TataNld','internet_top_pop/Telcove','internet_top_pop/Telecomserbia','internet_top_pop/Tinet','internet_top_pop/TLex','internet_top_pop/Tw','internet_top_pop/Twaren','internet_top_pop/Ulaknet','internet_top_pop/UniC','internet_top_pop/Uninet','internet_top_pop/Uninett2010','internet_top_pop/Uninett2011','internet_top_pop/Uran','internet_top_pop/UsCarrier','internet_top_pop/UsSignal','internet_top_pop/Uunet','internet_top_pop/Vinaren','internet_top_pop/VisionNet','internet_top_pop/VtlWavenet2008','internet_top_pop/VtlWavenet2011','internet_top_pop/WideJpn','internet_top_pop/Xeex','internet_top_pop/Xspedius','internet_top_pop/York','internet_top_pop/Zamren',
    'copenhagen/sms','copenhagen/fb_friends','copenhagen/calls','copenhagen/bt',
    'yeast_transcription','uni_email','euroroad','faa_routes','crime','polblogs','collins_yeast','interactome_stelzl',
    'interactome_yeast','plant_pol_robertson',
    'kegg_metabolic/aae','kegg_metabolic/afu','kegg_metabolic/ana','kegg_metabolic/ape','kegg_metabolic/atc','kegg_metabolic/ath','kegg_metabolic/atu','kegg_metabolic/bas','kegg_metabolic/bbu','kegg_metabolic/bha','kegg_metabolic/bja','kegg_metabolic/blo','kegg_metabolic/bme','kegg_metabolic/bms','kegg_metabolic/bsu','kegg_metabolic/buc','kegg_metabolic/cac','kegg_metabolic/ccr','kegg_metabolic/cef','kegg_metabolic/cel','kegg_metabolic/cgl','kegg_metabolic/cje','kegg_metabolic/cmu','kegg_metabolic/cpa','kegg_metabolic/cpe','kegg_metabolic/cpj','kegg_metabolic/cpn','kegg_metabolic/cte','kegg_metabolic/ctr','kegg_metabolic/dme','kegg_metabolic/dra','kegg_metabolic/ecc','kegg_metabolic/ece','kegg_metabolic/ecj','kegg_metabolic/eco','kegg_metabolic/ecs','kegg_metabolic/fnu','kegg_metabolic/hal','kegg_metabolic/hin','kegg_metabolic/hpj','kegg_metabolic/hpy','kegg_metabolic/hsa','kegg_metabolic/lil','kegg_metabolic/lin','kegg_metabolic/lla','kegg_metabolic/lmo','kegg_metabolic/mac','kegg_metabolic/mge','kegg_metabolic/mja','kegg_metabolic/mka','kegg_metabolic/mle','kegg_metabolic/mlo','kegg_metabolic/mma','kegg_metabolic/mmu','kegg_metabolic/mpe','kegg_metabolic/mpn','kegg_metabolic/mpu','kegg_metabolic/mtc','kegg_metabolic/mth','kegg_metabolic/mtu','kegg_metabolic/nma','kegg_metabolic/nme','kegg_metabolic/oih','kegg_metabolic/pab','kegg_metabolic/pae','kegg_metabolic/pai','kegg_metabolic/pfu','kegg_metabolic/pho','kegg_metabolic/pmu','kegg_metabolic/rco','kegg_metabolic/rno','kegg_metabolic/rpr','kegg_metabolic/rso','kegg_metabolic/sag','kegg_metabolic/sam','kegg_metabolic/san','kegg_metabolic/sau','kegg_metabolic/sav','kegg_metabolic/sce','kegg_metabolic/sco','kegg_metabolic/sfl','kegg_metabolic/sme','kegg_metabolic/smu','kegg_metabolic/son','kegg_metabolic/spg','kegg_metabolic/spm','kegg_metabolic/spn','kegg_metabolic/spo','kegg_metabolic/spr','kegg_metabolic/spy','kegg_metabolic/sso','kegg_metabolic/stm','kegg_metabolic/sto','kegg_metabolic/sty','kegg_metabolic/syn','kegg_metabolic/tac','kegg_metabolic/tel','kegg_metabolic/tma','kegg_metabolic/tpa','kegg_metabolic/tte','kegg_metabolic/tvo','kegg_metabolic/uur','kegg_metabolic/vch','kegg_metabolic/wbr','kegg_metabolic/xac','kegg_metabolic/xcc','kegg_metabolic/xfa','kegg_metabolic/ype','kegg_metabolic/ypk',
    'board_directors/net2m_2002-05-01','board_directors/net1m_2002-05-01','board_directors/net2m_2002-06-01','board_directors/net1m_2002-06-01','board_directors/net2m_2002-07-01','board_directors/net1m_2002-07-01','board_directors/net2m_2002-08-01','board_directors/net1m_2002-08-01','board_directors/net2m_2002-09-01','board_directors/net1m_2002-09-01','board_directors/net2m_2002-10-01','board_directors/net1m_2002-10-01','board_directors/net2m_2002-11-01','board_directors/net1m_2002-11-01','board_directors/net2m_2002-12-01','board_directors/net1m_2002-12-01','board_directors/net2m_2003-01-01','board_directors/net1m_2003-01-01','board_directors/net2m_2003-02-01','board_directors/net1m_2003-02-01','board_directors/net2m_2003-03-01','board_directors/net1m_2003-03-01','board_directors/net2m_2003-04-01','board_directors/net1m_2003-04-01','board_directors/net2m_2003-05-01','board_directors/net1m_2003-05-01','board_directors/net2m_2003-06-01','board_directors/net1m_2003-06-01','board_directors/net2m_2003-07-01','board_directors/net1m_2003-07-01','board_directors/net2m_2003-08-01','board_directors/net1m_2003-08-01','board_directors/net2m_2003-09-01','board_directors/net1m_2003-09-01','board_directors/net2m_2003-10-01','board_directors/net1m_2003-10-01','board_directors/net2m_2003-11-01','board_directors/net1m_2003-11-01','board_directors/net2m_2003-12-01','board_directors/net1m_2003-12-01','board_directors/net2m_2004-01-01','board_directors/net1m_2004-01-01','board_directors/net2m_2004-02-01','board_directors/net1m_2004-02-01','board_directors/net2m_2004-03-01','board_directors/net1m_2004-03-01','board_directors/net2m_2004-04-01','board_directors/net1m_2004-04-01','board_directors/net2m_2004-05-01','board_directors/net1m_2004-05-01','board_directors/net2m_2004-06-01','board_directors/net1m_2004-06-01','board_directors/net2m_2004-07-01','board_directors/net1m_2004-07-01','board_directors/net2m_2004-08-01','board_directors/net1m_2004-08-01','board_directors/net2m_2004-09-01','board_directors/net1m_2004-09-01','board_directors/net2m_2004-10-01','board_directors/net1m_2004-10-01','board_directors/net2m_2004-11-01','board_directors/net1m_2004-11-01','board_directors/net2m_2004-12-01','board_directors/net1m_2004-12-01','board_directors/net2m_2005-01-01','board_directors/net1m_2005-01-01','board_directors/net2m_2005-02-01','board_directors/net1m_2005-02-01','board_directors/net2m_2005-03-01','board_directors/net1m_2005-03-01','board_directors/net2m_2005-04-01','board_directors/net1m_2005-04-01','board_directors/net2m_2005-05-01','board_directors/net1m_2005-05-01','board_directors/net2m_2005-06-01','board_directors/net1m_2005-06-01','board_directors/net2m_2005-07-01','board_directors/net1m_2005-07-01','board_directors/net2m_2005-08-01','board_directors/net1m_2005-08-01','board_directors/net2m_2005-09-01','board_directors/net1m_2005-09-01','board_directors/net2m_2005-10-01','board_directors/net1m_2005-10-01','board_directors/net2m_2005-11-01','board_directors/net1m_2005-11-01','board_directors/net2m_2005-12-01','board_directors/net1m_2005-12-01','board_directors/net2m_2006-01-01','board_directors/net1m_2006-01-01','board_directors/net2m_2006-02-01','board_directors/net1m_2006-02-01','board_directors/net2m_2006-03-01','board_directors/net1m_2006-03-01','board_directors/net2m_2006-04-01','board_directors/net1m_2006-04-01','board_directors/net2m_2006-05-01','board_directors/net1m_2006-05-01','board_directors/net2m_2006-06-01','board_directors/net1m_2006-06-01','board_directors/net2m_2006-07-01','board_directors/net1m_2006-07-01','board_directors/net2m_2006-08-01','board_directors/net1m_2006-08-01','board_directors/net2m_2006-09-01','board_directors/net1m_2006-09-01','board_directors/net2m_2006-10-01','board_directors/net1m_2006-10-01','board_directors/net2m_2006-11-01','board_directors/net1m_2006-11-01','board_directors/net2m_2006-12-01','board_directors/net1m_2006-12-01','board_directors/net2m_2007-01-01','board_directors/net1m_2007-01-01','board_directors/net2m_2007-02-01','board_directors/net1m_2007-02-01','board_directors/net2m_2007-03-01','board_directors/net1m_2007-03-01','board_directors/net2m_2007-04-01','board_directors/net1m_2007-04-01','board_directors/net2m_2007-05-01','board_directors/net1m_2007-05-01','board_directors/net2m_2007-06-01','board_directors/net1m_2007-06-01','board_directors/net2m_2007-07-01','board_directors/net1m_2007-07-01','board_directors/net2m_2007-08-01','board_directors/net1m_2007-08-01','board_directors/net2m_2007-09-01','board_directors/net1m_2007-09-01','board_directors/net2m_2007-10-01','board_directors/net1m_2007-10-01','board_directors/net2m_2007-11-01','board_directors/net1m_2007-11-01','board_directors/net2m_2007-12-01','board_directors/net1m_2007-12-01','board_directors/net2m_2008-01-01','board_directors/net1m_2008-01-01','board_directors/net2m_2008-02-01','board_directors/net1m_2008-02-01','board_directors/net2m_2008-03-01','board_directors/net1m_2008-03-01','board_directors/net2m_2008-04-01','board_directors/net1m_2008-04-01','board_directors/net2m_2008-05-01','board_directors/net1m_2008-05-01','board_directors/net2m_2008-06-01','board_directors/net1m_2008-06-01','board_directors/net2m_2008-07-01','board_directors/net1m_2008-07-01','board_directors/net2m_2008-08-01','board_directors/net1m_2008-08-01','board_directors/net2m_2008-09-01','board_directors/net1m_2008-09-01','board_directors/net2m_2008-10-01','board_directors/net1m_2008-10-01','board_directors/net2m_2008-11-01','board_directors/net1m_2008-11-01','board_directors/net2m_2008-12-01','board_directors/net1m_2008-12-01','board_directors/net2m_2009-01-01','board_directors/net1m_2009-01-01','board_directors/net2m_2009-02-01','board_directors/net1m_2009-02-01','board_directors/net2m_2009-03-01','board_directors/net1m_2009-03-01','board_directors/net2m_2009-04-01','board_directors/net1m_2009-04-01','board_directors/net2m_2009-05-01','board_directors/net1m_2009-05-01','board_directors/net2m_2009-06-01','board_directors/net1m_2009-06-01','board_directors/net2m_2009-07-01','board_directors/net1m_2009-07-01','board_directors/net2m_2009-08-01','board_directors/net1m_2009-08-01','board_directors/net2m_2009-09-01','board_directors/net1m_2009-09-01','board_directors/net2m_2009-10-01','board_directors/net1m_2009-10-01','board_directors/net2m_2009-11-01','board_directors/net1m_2009-11-01','board_directors/net2m_2009-12-01','board_directors/net1m_2009-12-01','board_directors/net2m_2010-01-01','board_directors/net1m_2010-01-01','board_directors/net2m_2010-02-01','board_directors/net1m_2010-02-01','board_directors/net2m_2010-03-01','board_directors/net1m_2010-03-01','board_directors/net2m_2010-04-01','board_directors/net1m_2010-04-01','board_directors/net2m_2010-05-01','board_directors/net1m_2010-05-01','board_directors/net2m_2010-06-01','board_directors/net1m_2010-06-01','board_directors/net2m_2010-07-01','board_directors/net1m_2010-07-01','board_directors/net2m_2010-08-01','board_directors/net1m_2010-08-01','board_directors/net2m_2010-09-01','board_directors/net1m_2010-09-01','board_directors/net2m_2010-10-01','board_directors/net1m_2010-10-01','board_directors/net2m_2010-11-01','board_directors/net1m_2010-11-01','board_directors/net2m_2010-12-01','board_directors/net1m_2010-12-01','board_directors/net2m_2011-01-01','board_directors/net1m_2011-01-01','board_directors/net2m_2011-02-01','board_directors/net1m_2011-02-01','board_directors/net2m_2011-03-01','board_directors/net1m_2011-03-01','board_directors/net2m_2011-04-01','board_directors/net1m_2011-04-01','board_directors/net2m_2011-05-01','board_directors/net1m_2011-05-01','board_directors/net2m_2011-06-01','board_directors/net1m_2011-06-01','board_directors/net2m_2011-07-01','board_directors/net1m_2011-07-01','board_directors/net2m_2011-08-01','board_directors/net1m_2011-08-01',
    'dnc','interactome_figeys',
    'us_air_traffic',
    'urban_streets/ahmedabad','urban_streets/barcelona','urban_streets/bologna','urban_streets/brasilia','urban_streets/cairo','urban_streets/irvine1','urban_streets/irvine2','urban_streets/london','urban_streets/los-angeles','urban_streets/new-delhi','urban_streets/new-york','urban_streets/paris','urban_streets/richmond','urban_streets/san-francisco','urban_streets/savannah','urban_streets/seoul','urban_streets/venice','urban_streets/vienna','urban_streets/walnut-creek','urban_streets/washington',
    'interactome_vidal','power',
    'facebook_organizations/S1','facebook_organizations/S2','facebook_organizations/M1','facebook_organizations/M2','facebook_organizations/L1','facebook_organizations/L2',
    'fullerene_structures/C1500','fullerene_structures/C180','fullerene_structures/C2160','fullerene_structures/C240','fullerene_structures/C260','fullerene_structures/C2940','fullerene_structures/C320','fullerene_structures/C3840','fullerene_structures/C4860','fullerene_structures/C500','fullerene_structures/C540','fullerene_structures/C6000','fullerene_structures/C60','fullerene_structures/C720','fullerene_structures/C960',
    'jung',
    'celegans_interactomes/wi2007','celegans_interactomes/wi2004','celegans_interactomes/BPmaps','celegans_interactomes/LCI','celegans_interactomes/Microarray','celegans_interactomes/Phenotypes','celegans_interactomes/Interolog','celegans_interactomes/Genetic','celegans_interactomes/WI8','celegans_interactomes/IntegratedNetwork',
    'reactome','foursquare/NYC_restaurant_checkin','foursquare/NYC_restaurant_tips','jdk',
    'route_views/19971108','route_views/19971109','route_views/19971110','route_views/19971111','route_views/19971112','route_views/19971113','route_views/19971114','route_views/19971115','route_views/19971116','route_views/19971117','route_views/19971118','route_views/19971119','route_views/19971120','route_views/19971121','route_views/19971122','route_views/19971123','route_views/19971124','route_views/19971125','route_views/19971126','route_views/19971127','route_views/19971128','route_views/19971129','route_views/19971130','route_views/19971201','route_views/19971202','route_views/19971203','route_views/19971204','route_views/19971205','route_views/19971206','route_views/19971207','route_views/19971208','route_views/19971209','route_views/19971210','route_views/19971211','route_views/19971212','route_views/19971213','route_views/19971214','route_views/19971215','route_views/19971216','route_views/19971217','route_views/19971218','route_views/19971219','route_views/19971220','route_views/19971221','route_views/19971222','route_views/19971223','route_views/19971224','route_views/19971225','route_views/19971226','route_views/19971227','route_views/19971228','route_views/19971229','route_views/19971230','route_views/19971231','route_views/19980101','route_views/19980102','route_views/19980103','route_views/19980104','route_views/19980105','route_views/19980106','route_views/19980107','route_views/19980108','route_views/19980109','route_views/19980110','route_views/19980111','route_views/19980112','route_views/19980113','route_views/19980114','route_views/19980115','route_views/19980116','route_views/19980117','route_views/19980118','route_views/19980119','route_views/19980120','route_views/19980121','route_views/19980122','route_views/19980123','route_views/19980124','route_views/19980125','route_views/19980126','route_views/19980127','route_views/19980128','route_views/19980129','route_views/19980130','route_views/19980131','route_views/19980201','route_views/19980202','route_views/19980203','route_views/19980204','route_views/19980205','route_views/19980206','route_views/19980208','route_views/19980209','route_views/19980210','route_views/19980211','route_views/19980212','route_views/19980213','route_views/19980214','route_views/19980215','route_views/19980216','route_views/19980217','route_views/19980218','route_views/19980219','route_views/19980220','route_views/19980221','route_views/19980222','route_views/19980223','route_views/19980224','route_views/19980225','route_views/19980226','route_views/19980227','route_views/19980228','route_views/19980301','route_views/19980302','route_views/19980303','route_views/19980304','route_views/19980305','route_views/19980306','route_views/19980307','route_views/19980308','route_views/19980309','route_views/19980310','route_views/19980311','route_views/19980312','route_views/19980313','route_views/19980314','route_views/19980315','route_views/19980316','route_views/19980317','route_views/19980318','route_views/19980319','route_views/19980320','route_views/19980321','route_views/19980322','route_views/19980323','route_views/19980324','route_views/19980325','route_views/19980326','route_views/19980327','route_views/19980328','route_views/19980329','route_views/19980330','route_views/19980331','route_views/19980401','route_views/19980402','route_views/19980403','route_views/19980404','route_views/19980406','route_views/19980407','route_views/19980408','route_views/19980409','route_views/19980410','route_views/19980411','route_views/19980413','route_views/19980414','route_views/19980415','route_views/19980416','route_views/19980417','route_views/19980418','route_views/19980419','route_views/19980420','route_views/19980421','route_views/19980422','route_views/19980423','route_views/19980424','route_views/19980425','route_views/19980426','route_views/19980427','route_views/19980428','route_views/19980429','route_views/19980430','route_views/19980501','route_views/19980502','route_views/19980503','route_views/19980504','route_views/19980505','route_views/19980506','route_views/19980507','route_views/19980508','route_views/19980509','route_views/19980510','route_views/19980511','route_views/19980512','route_views/19980513','route_views/19980514','route_views/19980515','route_views/19980516','route_views/19980517','route_views/19980518','route_views/19980519','route_views/19980520','route_views/19980521','route_views/19980523','route_views/19980525','route_views/19980526','route_views/19980527','route_views/19980528','route_views/19980529','route_views/19980530','route_views/19980531','route_views/19980601','route_views/19980602','route_views/19980603','route_views/19980604','route_views/19980605','route_views/19980606','route_views/19980607','route_views/19980608','route_views/19980609','route_views/19980610','route_views/19980611','route_views/19980612','route_views/19980613','route_views/19980614','route_views/19980615','route_views/19980616','route_views/19980617','route_views/19980618','route_views/19980619','route_views/19980620','route_views/19980621','route_views/19980622','route_views/19980623','route_views/19980624','route_views/19980625','route_views/19980626','route_views/19980627','route_views/19980630','route_views/19980701','route_views/19980702','route_views/19980703','route_views/19980705','route_views/19980706','route_views/19980708','route_views/19980709','route_views/19980710','route_views/19980711','route_views/19980712','route_views/19980713','route_views/19980714','route_views/19980715','route_views/19980717','route_views/19980718','route_views/19980719','route_views/19980720','route_views/19980721','route_views/19980722','route_views/19980723','route_views/19980724','route_views/19980725','route_views/19980726','route_views/19980727','route_views/19980728','route_views/19980729','route_views/19980730','route_views/19980731','route_views/19980801','route_views/19980802','route_views/19980803','route_views/19980804','route_views/19980805','route_views/19980806','route_views/19980807','route_views/19980808','route_views/19980809','route_views/19980810','route_views/19980811','route_views/19980812','route_views/19980813','route_views/19980814','route_views/19980815','route_views/19980816','route_views/19980817','route_views/19980818','route_views/19980819','route_views/19980820','route_views/19980821','route_views/19980822','route_views/19980823','route_views/19980824','route_views/19980825','route_views/19980826','route_views/19980827','route_views/19980828','route_views/19980829','route_views/19980830','route_views/19980831','route_views/19980901','route_views/19980902','route_views/19980903','route_views/19980904','route_views/19980905','route_views/19980906','route_views/19980907','route_views/19980908','route_views/19980909','route_views/19980910','route_views/19980911','route_views/19980912','route_views/19980914','route_views/19980915','route_views/19980916','route_views/19980917','route_views/19980918','route_views/19980919','route_views/19980920','route_views/19980921','route_views/19980922','route_views/19980923','route_views/19980924','route_views/19980925','route_views/19980926','route_views/19980927','route_views/19980928','route_views/19980929','route_views/19980930','route_views/19981001','route_views/19981002','route_views/19981003','route_views/19981004','route_views/19981005','route_views/19981006','route_views/19981007','route_views/19981008','route_views/19981009','route_views/19981010','route_views/19981011','route_views/19981012','route_views/19981013','route_views/19981014','route_views/19981015','route_views/19981016','route_views/19981017','route_views/19981018','route_views/19981019','route_views/19981020','route_views/19981021','route_views/19981022','route_views/19981023','route_views/19981024','route_views/19981025','route_views/19981026','route_views/19981027','route_views/19981028','route_views/19981029','route_views/19981030','route_views/19981031','route_views/19981101','route_views/19981102','route_views/19981103','route_views/19981104','route_views/19981105','route_views/19981106','route_views/19981107','route_views/19981108','route_views/19981109','route_views/19981110','route_views/19981111','route_views/19981112','route_views/19981113','route_views/19981114','route_views/19981115','route_views/19981116','route_views/19981117','route_views/19981118','route_views/19981119','route_views/19981120','route_views/19981121','route_views/19981122','route_views/19981123','route_views/19981124','route_views/19981125','route_views/19981126','route_views/19981127','route_views/19981128','route_views/19981129','route_views/19981130','route_views/19981201','route_views/19981202','route_views/19981203','route_views/19981204','route_views/19981205','route_views/19981206','route_views/19981207','route_views/19981208','route_views/19981209','route_views/19981210','route_views/19981211','route_views/19981212','route_views/19981213','route_views/19981214','route_views/19981215','route_views/19981216','route_views/19981217','route_views/19981218','route_views/19981219','route_views/19981220','route_views/19981221','route_views/19981222','route_views/19981223','route_views/19981224','route_views/19981225','route_views/19981226','route_views/19981227','route_views/19981228','route_views/19981229','route_views/19981230','route_views/19981231','route_views/19990101','route_views/19990102','route_views/19990103','route_views/19990104','route_views/19990105','route_views/19990106','route_views/19990107','route_views/19990108','route_views/19990109','route_views/19990110','route_views/19990111','route_views/19990112','route_views/19990114','route_views/19990115','route_views/19990116','route_views/19990117','route_views/19990118','route_views/19990119','route_views/19990120','route_views/19990122','route_views/19990123','route_views/19990124','route_views/19990125','route_views/19990127','route_views/19990128','route_views/19990129','route_views/19990130','route_views/19990131','route_views/19990201','route_views/19990202','route_views/19990203','route_views/19990204','route_views/19990205','route_views/19990206','route_views/19990207','route_views/19990208','route_views/19990209','route_views/19990210','route_views/19990211','route_views/19990212','route_views/19990213','route_views/19990214','route_views/19990215','route_views/19990216','route_views/19990217','route_views/19990218','route_views/19990219','route_views/19990220','route_views/19990221','route_views/19990222','route_views/19990223','route_views/19990224','route_views/19990225','route_views/19990226','route_views/19990227','route_views/19990228','route_views/19990301','route_views/19990302','route_views/19990303','route_views/19990304','route_views/19990305','route_views/19990306','route_views/19990307','route_views/19990308','route_views/19990309','route_views/19990310','route_views/19990311','route_views/19990312','route_views/19990313','route_views/19990314','route_views/19990315','route_views/19990316','route_views/19990317','route_views/19990318','route_views/19990319','route_views/19990320','route_views/19990321','route_views/19990322','route_views/19990323','route_views/19990324','route_views/19990325','route_views/19990326','route_views/19990327','route_views/19990328','route_views/19990329','route_views/19990330','route_views/19990331','route_views/19990401','route_views/19990402','route_views/19990403','route_views/19990405','route_views/19990406','route_views/19990407','route_views/19990408','route_views/19990409','route_views/19990411','route_views/19990412','route_views/19990413','route_views/19990414','route_views/19990415','route_views/19990416','route_views/19990417','route_views/19990418','route_views/19990419','route_views/19990420','route_views/19990421','route_views/19990422','route_views/19990423','route_views/19990424','route_views/19990425','route_views/19990426','route_views/19990427','route_views/19990428','route_views/19990429','route_views/19990430','route_views/19990501','route_views/19990502','route_views/19990503','route_views/19990504','route_views/19990505','route_views/19990506','route_views/19990507','route_views/19990508','route_views/19990509','route_views/19990510','route_views/19990511','route_views/19990512','route_views/19990513','route_views/19990514','route_views/19990515','route_views/19990516','route_views/19990517','route_views/19990518','route_views/19990519','route_views/19990521','route_views/19990522','route_views/19990523','route_views/19990524','route_views/19990525','route_views/19990526','route_views/19990527','route_views/19990528','route_views/19990529','route_views/19990530','route_views/19990531','route_views/19990601','route_views/19990602','route_views/19990603','route_views/19990604','route_views/19990605','route_views/19990606','route_views/19990607','route_views/19990608','route_views/19990609','route_views/19990610','route_views/19990611','route_views/19990612','route_views/19990613','route_views/19990614','route_views/19990615','route_views/19990616','route_views/19990617','route_views/19990618','route_views/19990619','route_views/19990620','route_views/19990621','route_views/19990622','route_views/19990624','route_views/19990625','route_views/19990626','route_views/19990627','route_views/19990628','route_views/19990629','route_views/19990630','route_views/19990701','route_views/19990702','route_views/19990703','route_views/19990704','route_views/19990705','route_views/19990706','route_views/19990707','route_views/19990708','route_views/19990709','route_views/19990710','route_views/19990711','route_views/19990713','route_views/19990714','route_views/19990715','route_views/19990716','route_views/19990717','route_views/19990718','route_views/19990719','route_views/19990720','route_views/19990721','route_views/19990722','route_views/19990723','route_views/19990724','route_views/19990726','route_views/19990727','route_views/19990728','route_views/19990729','route_views/19990730','route_views/19990731','route_views/19990801','route_views/19990802','route_views/19990804','route_views/19990805','route_views/19990806','route_views/19990807','route_views/19990808','route_views/19990809','route_views/19990810','route_views/19990811','route_views/19990812','route_views/19990813','route_views/19990814','route_views/19990815','route_views/19990816','route_views/19990817','route_views/19990818','route_views/19990819','route_views/19990820','route_views/19990821','route_views/19990822','route_views/19990823','route_views/19990824','route_views/19990825','route_views/19990826','route_views/19990827','route_views/19990828','route_views/19990829','route_views/19990830','route_views/19990831','route_views/19990901','route_views/19990902','route_views/19990903','route_views/19990904','route_views/19990905','route_views/19990906','route_views/19990907','route_views/19990908','route_views/19990909','route_views/19990910','route_views/19990911','route_views/19990912','route_views/19990913','route_views/19990914','route_views/19990915','route_views/19990916','route_views/19990917','route_views/19990918','route_views/19990919','route_views/19990920','route_views/19990921','route_views/19990922','route_views/19990923','route_views/19990924','route_views/19990925','route_views/19990926','route_views/19990927','route_views/19990928','route_views/19990929','route_views/19990930','route_views/19991001','route_views/19991002','route_views/19991003','route_views/19991004','route_views/19991005','route_views/19991006','route_views/19991007','route_views/19991008','route_views/19991009','route_views/19991010','route_views/19991011','route_views/19991013','route_views/19991016','route_views/19991017','route_views/19991018','route_views/19991019','route_views/19991026','route_views/19991027','route_views/19991028','route_views/19991031','route_views/19991105','route_views/19991108','route_views/19991109','route_views/19991110','route_views/19991111','route_views/19991112','route_views/19991113','route_views/19991114','route_views/19991115','route_views/19991116','route_views/19991118','route_views/19991121','route_views/19991122','route_views/19991204','route_views/19991205','route_views/19991206','route_views/19991208','route_views/19991209','route_views/19991210','route_views/19991211','route_views/19991212','route_views/19991213','route_views/19991214','route_views/19991215','route_views/19991216','route_views/19991217','route_views/19991218','route_views/19991219','route_views/19991220','route_views/19991221','route_views/19991222','route_views/19991223','route_views/19991224','route_views/19991225','route_views/19991226','route_views/19991227','route_views/19991228','route_views/19991229','route_views/19991230','route_views/19991231','route_views/20000101','route_views/20000102',
    'software_dependencies/jdk','software_dependencies/jung-j','software_dependencies/javax','software_dependencies/java','software_dependencies/slucene','software_dependencies/weka','software_dependencies/jung-c','software_dependencies/guava','software_dependencies/org','software_dependencies/colt','software_dependencies/jung','software_dependencies/sjung','software_dependencies/flamingo','software_dependencies/scolt','software_dependencies/jmail','software_dependencies/sjbullet','software_dependencies/junit',
    'sp_infectious',
    'word_adjacency/darwin','word_adjacency/french','word_adjacency/spanish','word_adjacency/japanese',
    'dblp_cite','anybeat','chicago_road','google',
    ##'genetic_multiplex',multilayer
    'marvel_universe','internet_as','word_assoc','cora','movielens_100k',
    'caida_as/20071112','caida_as/20071105','caida_as/20071029','caida_as/20071022','caida_as/20071015','caida_as/20071008','caida_as/20071001','caida_as/20070924','caida_as/20070917','caida_as/20070910','caida_as/20070903','caida_as/20070827','caida_as/20070820','caida_as/20070813','caida_as/20070806','caida_as/20070730','caida_as/20070723','caida_as/20070716','caida_as/20070709','caida_as/20070702','caida_as/20070625','caida_as/20070618','caida_as/20070611','caida_as/20070604','caida_as/20070528','caida_as/20070521','caida_as/20070514','caida_as/20070507','caida_as/20070430','caida_as/20070423','caida_as/20070416','caida_as/20070409','caida_as/20070402','caida_as/20070326','caida_as/20070319','caida_as/20070312','caida_as/20070305','caida_as/20070226','caida_as/20070219','caida_as/20070212','caida_as/20070205','caida_as/20070129','caida_as/20070122','caida_as/20070115','caida_as/20070108','caida_as/20070101','caida_as/20061225','caida_as/20061218','caida_as/20061211','caida_as/20061204','caida_as/20061127','caida_as/20061120','caida_as/20061113','caida_as/20061106','caida_as/20061030','caida_as/20061023','caida_as/20061016','caida_as/20061009','caida_as/20061002','caida_as/20060925','caida_as/20060918','caida_as/20060911','caida_as/20060904','caida_as/20060828','caida_as/20060821','caida_as/20060814','caida_as/20060807','caida_as/20060731','caida_as/20060724','caida_as/20060717','caida_as/20060710','caida_as/20060703','caida_as/20060626','caida_as/20060619','caida_as/20060612','caida_as/20060605','caida_as/20060529','caida_as/20060522','caida_as/20060515','caida_as/20060508','caida_as/20060501','caida_as/20060424','caida_as/20060417','caida_as/20060410','caida_as/20060403','caida_as/20060327','caida_as/20060320','caida_as/20060313','caida_as/20060306','caida_as/20060227','caida_as/20060220','caida_as/20060213','caida_as/20060206','caida_as/20060130','caida_as/20060123','caida_as/20060116','caida_as/20060109','caida_as/20060102','caida_as/20051205','caida_as/20051107','caida_as/20051003','caida_as/20050905','caida_as/20050801','caida_as/20050704','caida_as/20050606','caida_as/20050502','caida_as/20050404','caida_as/20050307','caida_as/20050207','caida_as/20050103','caida_as/20041206','caida_as/20041101','caida_as/20041004','caida_as/20040906','caida_as/20040802','caida_as/20040705','caida_as/20040607','caida_as/20040503','caida_as/20040405','caida_as/20040301','caida_as/20040202','caida_as/20040105',
    'digg_reply','nematode_mammal',
    'linux',
    'arxiv_citation/HepPh','arxiv_citation/HepTh',
    'scotus_majority/2008','scotus_majority/2007','topology',
    #'hiv_transmission',restricted access
    'email_enron','inploid','pgp_strong',
    'paris_transportation','slashdot_threads','python_dependency',
    'gnutella/04','gnutella/06','gnutella/08','gnutella/09','gnutella/24','gnutella/25','gnutella/30','gnutella/31',
    'lkml_reply','marker_cafe',
    'epinions_trust','twitter_15m','prosper','wiki_link_dyn','livemocha',
    'foursquare_friendships/old','foursquare_friendships/new',
    'arxiv_authors/AstroPh','arxiv_authors/CondMat','arxiv_authors/GrQc','arxiv_authors/HepPh','arxiv_authors/HepTh',
    'dbpedia_writer','lastfm_aminer','wordnet','dbtropes_feature','douban','dbpedia_starring',
    'github','dbpedia_recordlabel','dbpedia_producer','academia_edu','google_plus','flickr_aminer',
    'dbpedia_location','dbpedia_occupation','email_eu','dbpedia_genre','discogs_label','stanford_web',
    'notre_dame_web','corporate_directors',
    ##'openstreetmap', 90K networs!
    'lkml_thread','citeseer',
    'amazon_copurchases/302','amazon_copurchases/312','amazon_copurchases/505','amazon_copurchases/601',
    'dblp_coauthor_snap','bookcrossing','twitter','flickr_groups','visualizeus','dbpedia_country','petster',
    'stackoverflow','yahoo_ads','berkstan_web','myspace_aminer','citeulike','google_web','dbpedia_team',
    'bibsonomy','reuters',
    'lastfm/band','lastfm/song',
    'wikitree','hyves','trec_web','as_skitter','trec','discogs_genre',
    'dblp_coauthor',
    'wikipedia_growth',
    'roadnet/CA','roadnet/PA','roadnet/TX',
    'discogs_affiliation','wiki_categories','baidu',
    'twitter_events/NYClimateMarch2014','twitter_events/Cannes2013','twitter_events/MoscowAthletics2013','twitter_events/MLKing2013','twitter_events/ObamaInIsrael2013',
    'flickr_growth','wikipedia-en-talk','flixster',
    'qa_user/stackoverflow_all','qa_user/stackoverflow_a2q','qa_user/stackoverflow_c2q','qa_user/stackoverflow_c2a','qa_user/mathoverflow_all','qa_user/mathoverflow_a2q','qa_user/mathoverflow_c2q','qa_user/mathoverflow_c2a','qa_user/superuser_all','qa_user/superuser_a2q','qa_user/superuser_c2q','qa_user/superuser_c2a','qa_user/askubuntu_all','qa_user/askubuntu_a2q','qa_user/askubuntu_c2q','qa_user/askubuntu_c2a',
    ##'mag_history_coauthor','mag_geology_coauthor', these could be weighted
    'wiki_talk/ar','wiki_talk/bn','wiki_talk/br','wiki_talk/ca','wiki_talk/cy','wiki_talk/de','wiki_talk/el','wiki_talk/en','wiki_talk/eo','wiki_talk/es','wiki_talk/eu','wiki_talk/fr','wiki_talk/gl','wiki_talk/ht','wiki_talk/it','wiki_talk/ja','wiki_talk/lv','wiki_talk/nds','wiki_talk/nl','wiki_talk/oc','wiki_talk/pl','wiki_talk/pt','wiki_talk/ru','wiki_talk/sk','wiki_talk/sr','wiki_talk/sv','wiki_talk/vi','wiki_talk/zh',
    'livejournal_aminer',
    #'us_patents', error
    'foursquare_global','dbpedia_all',
    ##'livejournal',errore
    'mislove_osn/flickr','mislove_osn/livejournal','mislove_osn/orkut','mislove_osn/youtube',
    'twitter_sample',
    'dblp_simplices/max-25-bip','dblp_simplices/max-25-proj','dblp_simplices/full-bip','dblp_simplices/full-proj',
    'bitcoin','dblp_author_paper',
    'wikipedia_link/af','wikipedia_link/als','wikipedia_link/am','wikipedia_link/an','wikipedia_link/ar','wikipedia_link/arz','wikipedia_link/as','wikipedia_link/ast','wikipedia_link/az','wikipedia_link/ba','wikipedia_link/bar','wikipedia_link/bat_smg','wikipedia_link/bcl','wikipedia_link/be','wikipedia_link/be_x_old','wikipedia_link/bg','wikipedia_link/bh','wikipedia_link/bn','wikipedia_link/bpy','wikipedia_link/br','wikipedia_link/bs','wikipedia_link/bug','wikipedia_link/ca','wikipedia_link/cdo','wikipedia_link/ce','wikipedia_link/ceb','wikipedia_link/ckb','wikipedia_link/co','wikipedia_link/crh','wikipedia_link/cs','wikipedia_link/csb','wikipedia_link/cv','wikipedia_link/cy','wikipedia_link/da','wikipedia_link/de','wikipedia_link/diq','wikipedia_link/dv','wikipedia_link/el','wikipedia_link/eml','wikipedia_link/en','wikipedia_link/eo','wikipedia_link/es','wikipedia_link/et','wikipedia_link/eu','wikipedia_link/fa','wikipedia_link/fi','wikipedia_link/fiu_vro','wikipedia_link/fo','wikipedia_link/fr','wikipedia_link/fy','wikipedia_link/ga','wikipedia_link/gag','wikipedia_link/gan','wikipedia_link/gd','wikipedia_link/gl','wikipedia_link/glk','wikipedia_link/gu','wikipedia_link/gv','wikipedia_link/hak','wikipedia_link/he','wikipedia_link/hi','wikipedia_link/hif','wikipedia_link/hr','wikipedia_link/hsb','wikipedia_link/ht','wikipedia_link/hu','wikipedia_link/hy','wikipedia_link/ia','wikipedia_link/id','wikipedia_link/ilo','wikipedia_link/io','wikipedia_link/is','wikipedia_link/it','wikipedia_link/ja','wikipedia_link/jv','wikipedia_link/ka','wikipedia_link/kk','wikipedia_link/kn','wikipedia_link/ko','wikipedia_link/ku','wikipedia_link/kv','wikipedia_link/ky','wikipedia_link/la','wikipedia_link/lb','wikipedia_link/lez','wikipedia_link/li','wikipedia_link/lmo','wikipedia_link/lo','wikipedia_link/lt','wikipedia_link/lv','wikipedia_link/map_bms','wikipedia_link/mg','wikipedia_link/mhr','wikipedia_link/mi','wikipedia_link/min','wikipedia_link/mk','wikipedia_link/ml','wikipedia_link/mn','wikipedia_link/mr','wikipedia_link/ms','wikipedia_link/my','wikipedia_link/myv','wikipedia_link/mzn','wikipedia_link/nah','wikipedia_link/nap','wikipedia_link/nds','wikipedia_link/nds_nl','wikipedia_link/ne','wikipedia_link/new','wikipedia_link/nl','wikipedia_link/nn','wikipedia_link/no','wikipedia_link/nso','wikipedia_link/oc','wikipedia_link/or','wikipedia_link/os','wikipedia_link/pa','wikipedia_link/pam','wikipedia_link/pl','wikipedia_link/pms','wikipedia_link/pnb','wikipedia_link/ps','wikipedia_link/pt','wikipedia_link/qu','wikipedia_link/ro','wikipedia_link/roa_tara','wikipedia_link/ru','wikipedia_link/rue','wikipedia_link/sa','wikipedia_link/sah','wikipedia_link/scn','wikipedia_link/sco','wikipedia_link/se','wikipedia_link/sh','wikipedia_link/si','wikipedia_link/simple','wikipedia_link/sk','wikipedia_link/sl','wikipedia_link/so','wikipedia_link/sq','wikipedia_link/sr','wikipedia_link/su','wikipedia_link/sv','wikipedia_link/sw','wikipedia_link/ta','wikipedia_link/te','wikipedia_link/tg','wikipedia_link/th','wikipedia_link/tk','wikipedia_link/tl','wikipedia_link/tr','wikipedia_link/tt','wikipedia_link/uk','wikipedia_link/ur','wikipedia_link/uz','wikipedia_link/vec','wikipedia_link/vi','wikipedia_link/vls','wikipedia_link/vo','wikipedia_link/wa','wikipedia_link/war','wikipedia_link/xal','wikipedia_link/xmf','wikipedia_link/yi','wikipedia_link/yo','wikipedia_link/zh','wikipedia_link/zh_min_nan','wikipedia_link/zh_yue',
    'dbpedia_link',
    #'us_roads/AK','us_roads/AL','us_roads/AR','us_roads/AZ','us_roads/CA','us_roads/CO','us_roads/CT','us_roads/DC','us_roads/DE','us_roads/FL','us_roads/GA','us_roads/HI','us_roads/IA','us_roads/ID','us_roads/IL','us_roads/IN','us_roads/KS','us_roads/KY','us_roads/LA','us_roads/MA','us_roads/MD','us_roads/ME','us_roads/MI','us_roads/MN','us_roads/MO','us_roads/MS','us_roads/MT','us_roads/NC','us_roads/ND','us_roads/NE','us_roads/NH','us_roads/NJ','us_roads/NM','us_roads/NV','us_roads/NY','us_roads/OH','us_roads/OK','us_roads/OR','us_roads/PA','us_roads/RI','us_roads/SC','us_roads/SD','us_roads/TN','us_roads/TX','us_roads/UT','us_roads/VA','us_roads/VT','us_roads/WA','us_roads/WI','us_roads/WV','us_roads/WY','us_roads/merged',  BIG
    'delicious/ut','delicious/ti','delicious/ui',
    'trackers','twitter_social','twitter_2009',
    'soc_net_comms/friendster','soc_net_comms/livejournal','soc_net_comms/orkut','soc_net_comms/youtube','soc_net_comms/dblp','soc_net_comms/amazon' #BIG
        # PROBLEMS 'ego_social/facebook_0','ego_social/facebook_107','ego_social/facebook_348','ego_social/facebook_414','ego_social/facebook_686','ego_social/facebook_698','ego_social/facebook_1684','ego_social/facebook_1912','ego_social/facebook_3437','ego_social/facebook_3980','ego_social/facebook_combined','ego_social/gplus_100129275726588145876','ego_social/gplus_100329698645326486178','ego_social/gplus_100466178325794757407','ego_social/gplus_100500197140377336562','ego_social/gplus_100518419853963396365','ego_social/gplus_100521671383026672718','ego_social/gplus_100535338638690515335','ego_social/gplus_100637660947564674695','ego_social/gplus_100668989009254813743','ego_social/gplus_100715738096376666180','ego_social/gplus_100720409235366385249','ego_social/gplus_100962871525684315897','ego_social/gplus_101130571432010257170','ego_social/gplus_101133961721621664586','ego_social/gplus_101185748996927059931','ego_social/gplus_101263615503715477581','ego_social/gplus_101373961279443806744','ego_social/gplus_101499880233887429402','ego_social/gplus_101541879642294398860','ego_social/gplus_101560853443212199687','ego_social/gplus_101626577406833098387','ego_social/gplus_101848191156408080085','ego_social/gplus_101997124338642780860','ego_social/gplus_102170431816592344972','ego_social/gplus_102340116189726655233','ego_social/gplus_102615863344410467759','ego_social/gplus_102778563580121606331','ego_social/gplus_103236949470535942612','ego_social/gplus_103241736833663734962','ego_social/gplus_103251633033550231172','ego_social/gplus_103338524411980406972','ego_social/gplus_103503116383846951534','ego_social/gplus_103537112468125883734','ego_social/gplus_103752943025677384806','ego_social/gplus_103892332449873403244','ego_social/gplus_104076158580173410325','ego_social/gplus_104105354262797387583','ego_social/gplus_104226133029319075907','ego_social/gplus_104290609881668164623','ego_social/gplus_104607825525972194062','ego_social/gplus_104672614700283598130','ego_social/gplus_104905626100400792399','ego_social/gplus_104917160754181459072','ego_social/gplus_104987932455782713675','ego_social/gplus_105565257978663183206','ego_social/gplus_105646458226420473639','ego_social/gplus_106186407539128840569','ego_social/gplus_106228758905254036967','ego_social/gplus_106328207304735502636','ego_social/gplus_106382433884876652170','ego_social/gplus_106417861423111072106','ego_social/gplus_106724181552911298818','ego_social/gplus_106837574755355833243','ego_social/gplus_107013688749125521109','ego_social/gplus_107040353898400532534','ego_social/gplus_107203023379915799071','ego_social/gplus_107223200089245371832','ego_social/gplus_107296660002634487593','ego_social/gplus_107362628080904735459','ego_social/gplus_107459220492917008623','ego_social/gplus_107489144252174167638','ego_social/gplus_107965826228461029730','ego_social/gplus_108156134340151350951','ego_social/gplus_108404515213153345305','ego_social/gplus_108541235642523883716','ego_social/gplus_108883879052307976051','ego_social/gplus_109130886479781915270','ego_social/gplus_109213135085178239952','ego_social/gplus_109327480479767108490','ego_social/gplus_109342148209917802565','ego_social/gplus_109596373340495798827','ego_social/gplus_109602109099036550366','ego_social/gplus_110232479818136355682','ego_social/gplus_110241952466097562819','ego_social/gplus_110538600381916983600','ego_social/gplus_110581012109008817546','ego_social/gplus_110614416163543421878','ego_social/gplus_110701307803962595019','ego_social/gplus_110739220927723360152','ego_social/gplus_110809308822849680310','ego_social/gplus_110971010308065250763','ego_social/gplus_111048918866742956374','ego_social/gplus_111058843129764709244','ego_social/gplus_111091089527727420853','ego_social/gplus_111213696402662884531','ego_social/gplus_112317819390625199896','ego_social/gplus_112463391491520264813','ego_social/gplus_112573107772208475213','ego_social/gplus_112724573277710080670','ego_social/gplus_112737356589974073749','ego_social/gplus_112787435697866537461','ego_social/gplus_113112256846010263985','ego_social/gplus_113122049849685469495','ego_social/gplus_113171096418029011322','ego_social/gplus_113356364521839061717','ego_social/gplus_113455290791279442483','ego_social/gplus_113597493946570654755','ego_social/gplus_113718775944980638561','ego_social/gplus_113799277735885972934','ego_social/gplus_113881433443048137993','ego_social/gplus_114054672576929802335','ego_social/gplus_114104634069486127920','ego_social/gplus_114122960748905067938','ego_social/gplus_114124942936679476879','ego_social/gplus_114147483140782280818','ego_social/gplus_114336431216099933033','ego_social/gplus_115121555137256496805','ego_social/gplus_115273860520983542999','ego_social/gplus_115360471097759949621','ego_social/gplus_115455024457484679647','ego_social/gplus_115516333681138986628','ego_social/gplus_115573545440464933254','ego_social/gplus_115576988435396060952','ego_social/gplus_115625564993990145546','ego_social/gplus_116059998563577101552','ego_social/gplus_116247667398036716276','ego_social/gplus_116315897040732668413','ego_social/gplus_116450966137824114154','ego_social/gplus_116807883656585676940','ego_social/gplus_116825083494890429556','ego_social/gplus_116899029375914044550','ego_social/gplus_116931379084245069738','ego_social/gplus_117412175333096244275','ego_social/gplus_117503822947457399073','ego_social/gplus_117668392750579292609','ego_social/gplus_117734260411963901771','ego_social/gplus_117798157258572080176','ego_social/gplus_117866881767579360121','ego_social/gplus_118107045405823607895','ego_social/gplus_118255645714452180374','ego_social/gplus_118379821279745746467','ego_social/gplus_combined',
    ]

path_g_drive="measureGraphs/"
measure_allgraphs(model_names,download_file,store_on_gdrive,path_g_drive,prefix_to_weight,"unweighted")

# If interested in drawing

In [ ]:
# from graph_tool.all import *
# g = collection.data["celegansneural"]
# state = minimize_nested_blockmodel_dl(g)
# state = minimize_nested_blockmodel_dl(g)
# state.draw()